Python imports

In [1]:
import implicit
import pandas as pd
from scipy import sparse
import numpy
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
from lightfm.cross_validation import random_train_test_split
from sklearn.externals import joblib

Read CSV File. It has a header with customerid,productid,amount

In [2]:
df = pd.read_csv('interactions.csv', usecols = ['customerid','productid'], dtype = {'customerid':str, 'productid':str})

Create a new LightFM Dataset containing the customer and product interactions

In [3]:
uniqueCustomer = df['customerid'].unique()
uniqueProducts = df['productid'].unique()

dataset = Dataset()
dataset.fit(uniqueCustomer,uniqueProducts)

(interactions, weights) = dataset.build_interactions((row.customerid, row.productid)
                                                      for index, row in df.iterrows())

print(repr(interactions))

    
print('Total number of unique customers : %.0f' % len(uniqueCustomer))
print('Total number of unique products  : %.0f' % len(uniqueProducts))


<996243x845209 sparse matrix of type '<class 'numpy.int32'>'
	with 10868978 stored elements in COOrdinate format>
Total number of unique customers : 996243
Total number of unique products  : 845209


Creating train and test split model

In [4]:
(train,test) = random_train_test_split(interactions, test_percentage=0.2)

Train the Implicit model

In [5]:
print('Training Implicit model')
implicitmodel = implicit.als.AlternatingLeastSquares(factors=16,use_gpu=False,num_threads=0,use_native=True)
%time implicitmodel.fit(train)

Training Implicit model


100%|██████████| 15.0/15 [01:28<00:00,  6.05s/it]

CPU times: user 4min 41s, sys: 4min 52s, total: 9min 33s
Wall time: 1min 30s


Train the LightFM model

In [6]:
print('Training LightFM model')
model = LightFM(loss='warp', no_components=16)
%time model.fit(train, epochs=100, num_threads=8)

Training LightFM model
CPU times: user 1h 28min 8s, sys: 3.35 s, total: 1h 28min 12s
Wall time: 13min 54s


In [7]:
joblib.dump(model, 'lightfmmodel.sav')
print('Model saved');

Model saved


Evaluate the model

In [8]:
#print('Evaluating model')

#aucscore = auc_score(model, interactions, num_threads=8).mean()
#print('AUC Score:          %.2f.' % (aucscore))

#aucscore_train = auc_score(model, train, num_threads=8).mean()
#print('AUC Score train:    %.2f.' % (aucscore_train))

#train_precision = precision_at_k(model, train, k=5, num_threads=8).mean()
#test_precision = precision_at_k(model, test, k=5, num_threads=8).mean()
#print('Precision@k5: train %.2f.' % (train_precision))
#print('Precision@k5: test  %.2f.' % (test_precision))

Sample predictions

In [10]:
def sample_recommendation(model, data, user_ids):

    user2idx, unused1, item2idx, unused2 = dataset.mapping()
    
    item2idx_inv = dict(map(reversed, item2idx.items()))    

    n_users, n_items = data.shape

    for user_id in user_ids:
        
        print('Recommentations for user %s' % user_id)
        
        scores = model.predict(user2idx[user_id], numpy.arange(n_items))
        for i in range(len(scores)):
            itemid = item2idx_inv[i]
            score = scores[i]
            
            if (score > 0.6):
                #print(' Item %s with Score %2f.' % (itemid, score))
                print('%s\t%2f' % (itemid, score))
        
sample_recommendation(model, interactions, ['31460292'])Recommentations for user 31460292
90057363	0.979922
48779055	1.219488
92336287	0.619047
47833922	0.841020
121090385	1.023198
138531879	1.583254
140554978	2.731774
5689172	1.688485
100540639	1.358565
139241288	1.292006
47878416	0.979951
64190301	0.945828
43972255	0.758818
138704636	2.302911
121623700	0.998329
84741712	1.492488
15053199	0.810409
29020424	1.736118
52928156	0.852643
140701434	1.352199
43972783	1.565849
42649865	0.772957
79409153	2.080463
16504414	1.021562
91960669	1.225156
140594566	1.417287
117276294	1.171021
50874070	1.068287
39188789	1.117866
113966530	1.169820
140772709	1.286793
139447140	1.996504
133631595	1.890002
46361715	0.880706
139270905	1.706772
44126449	1.032962
117501702	1.121705
140317791	2.370730
46324362	1.119576
13838562	1.812879
56306666	0.849693
98598818	1.909992
37944207	0.642718
140059254	0.621110
113945468	1.504893
40934755	1.872431
64481421	1.251896
92031643	1.520630
116765328	1.140778
40324555	0.785648
139566128	1.447769
16828487	1.007742
25207386	1.285042
46019764	0.639723
115084730	1.309370
113374527	0.917020
140903682	1.925845
138326629	0.632818
139711397	1.233795
103135018	1.034819
85749121	0.702862
120158578	0.740293
139218285	0.864655
139511664	2.581814
140495218	1.851636
117279165	1.238621
140172408	1.020172
141331904	0.882832
14303792	0.982296
139447128	1.870774
140155172	0.788239
102825601	1.662852
47947766	0.779748
87483776	2.338297
141686397	0.827704
1938455	1.860623
137412504	1.924158
139566136	1.195155
78341846	1.415018
96724696	1.534000
135202281	1.756574
139511619	2.442629
56166358	0.890006
75072676	0.829459
138273637	1.645060
135202288	1.732546
132866780	2.429721
43943028	1.998541
140919934	0.972382
21279267	0.760796
140155053	1.024899
40108511	1.473670
33641777	0.845202
141636007	1.209631
62180153	0.740431
64481467	1.747410
122560768	1.061970
44912332	1.375865
141300559	1.935914
92336295	0.642470
65245060	0.899158
89650662	0.824207
102077443	2.227695
40981527	1.815401
121624004	1.344943
116831274	0.928998
26214061	1.424996
89597446	1.341602
30625408	1.351950
140335506	1.431454
140179554	0.715597
140699002	1.190329
140163418	1.080676
83200102	0.939038
40891020	0.614500
45047262	1.518807
116765657	1.264194
6330462	0.873166
127880907	1.777929
66570679	1.322976
39293989	1.135122
44252949	0.851028
139722943	1.118871
138273719	0.979062
125178629	2.240006
91474632	0.822914
140156871	0.912362
47824079	0.738659
131933224	0.882406
126059933	1.374896
140556590	0.803365
21439212	0.929168
47937475	1.407435
45276329	1.107146
47719298	0.938920
78479679	0.659001
15149160	1.195513
47351775	2.174134
140497904	1.571846
34671082	1.542599
45546050	0.831351
61100828	0.639987
138273576	2.494501
52519203	0.620825
135757484	1.063750
136982456	1.410476
2829971	1.396361
103554411	1.090247
115314288	0.822072
90072104	1.656344
41798213	1.367731
140495404	0.886760
40893304	0.897656
140838603	1.148936
2937851	1.058525
140951383	0.750180
39233879	1.224040
42564746	1.186055
47877879	0.991559
3561211	0.762345
140919409	1.116668
47833924	1.072714
103554616	1.766660
13863234	0.946723
119673598	1.041116
115538398	0.793021
140336756	1.596864
115008899	1.007937
21281453	0.989647
93826682	0.944371
140701659	0.806173
88702119	0.934405
46388828	1.488588
73478866	0.768613
115122290	0.998949
140701673	0.847914
141375488	1.347326
33791016	0.753187
9185056	1.262107
43950695	0.639008
42451952	1.737058
46376216	1.218187
140702032	0.895189
41921817	1.429004
72654575	0.639549
47779369	0.966185
140315411	0.722929
139458458	0.983887
102974139	2.297577
3028258	0.763661
138705306	0.915306
45320696	1.638996
78492798	1.274883
139658785	0.696978
140563275	0.719784
45429925	0.884007
139972127	1.193421
45255241	1.991476
29048263	1.290978
139566133	1.061082
25782082	1.132432
128927698	1.529909
137807446	0.795939
103895990	0.918826
92031642	1.102619
141040616	1.431831
39151356	0.955214
45119672	0.903318
121376812	1.140004
127447115	1.186945
34975263	1.134278
139566132	0.666742
139631134	1.624975
72654579	0.766416
44337085	1.397599
57445682	0.887817
115855974	0.886428
48841820	0.935959
39143731	1.027567
65245138	0.712512
140672374	0.864741
121090320	1.749724
120439251	1.667878
119872872	0.732952
125166167	1.042049
72727038	1.688497
87933678	0.902421
20422262	0.879195
139218852	1.772749
71621806	1.279012
29861695	0.605859
120838057	1.233805
140155062	1.530857
22146913	0.782903
41016153	0.613242
117707922	2.105040
130548736	0.906494
140443447	1.696011
127553460	1.480408
105816871	1.217460
140177214	1.587153
137807344	0.904904
137995898	1.514540
140594588	0.673112
140509900	1.496706
45526853	1.031253
140315217	0.674161
37352403	1.079839
32162891	0.639928
29974749	1.675523
3070651	1.009513
140562622	0.928677
141289038	0.818034
44169696	1.524235
140489411	1.690332
88699168	1.250671
45526293	0.780075
84103785	2.212177
120837982	2.292469
40401987	0.809946
35053979	1.230852
1778730	2.226036
15500480	1.313116
70373501	0.948307
11387368	1.328480
139185718	1.386145
136985607	0.875872
42523951	1.422389
47873534	1.170917
1828092	0.823037
46303609	1.028978
141375486	1.395950
120838024	0.663615
46179340	0.984947
21382596	1.837340
94636035	1.082592
76667270	0.690789
135202368	1.705236
25425711	0.602678
68111046	1.064980
70425989	0.856079
115693747	0.689352
11323742	1.160622
32013874	1.022769
40418642	0.796624
42546391	1.235760
34464968	0.808694
140154077	0.757118
140897406	0.785029
140439688	1.054852
106869486	2.118432
140315255	0.841186
2896572	1.014549
41065363	1.193478
14051175	0.717844
141071464	0.691747
141527306	1.545847
25035463	1.405776
35374517	1.729408
140794654	0.837508
3086447	1.659101
30525861	1.123002
27182641	0.653256
73071241	0.708390
66569877	0.987500
14577621	0.743274
34092264	0.788633
35191482	0.732150
36263158	1.056107
38295574	0.930088
128548492	0.889023
40952657	0.604849
139970094	0.858306
15149414	0.830287
134568335	1.487575
118009335	0.938813
104279116	0.837832
56877137	1.266402
42893549	0.783761
47184244	0.869349
137405608	1.465829
44787468	1.416530
132142160	1.372551
3012777	2.214690
115538825	1.759378
92336299	0.925435
35384382	0.792817
30570168	1.114644
115538768	0.797660
139232888	0.998160
18938759	1.652406
42812079	0.849181
45252465	0.676838
140927177	1.101098
2940330	1.805609
28844959	1.074038
120982636	2.667280
69442648	0.640160
117181357	1.202646
116765642	0.670287
50201002	1.335879
44658996	0.962712
139652660	0.912454
114319656	1.191119
125451755	1.255926
139984509	2.466156
18706669	0.826216
139571966	1.319678
120599044	1.276586
2894768	0.734150
30553625	0.717546
43403521	1.923308
37846033	1.426007
47873332	1.702648
21108270	1.366798
83459289	1.180293
139566123	1.675217
30612702	2.517660
39163584	0.737144
122972267	1.957063
39535057	1.202496
45409695	1.948448
116765456	2.071623
83985469	0.605009
11396517	2.118795
64190226	1.630882
139235100	1.490152
36079806	0.652097
101359062	1.070548
15990610	1.448404
41515936	0.765737
92336309	0.899561
140222874	1.680362
46272765	1.753291
2964695	1.183653
33793670	1.290038
68908483	0.754367
94946756	0.747827
22326244	0.894597
69438655	0.690608
17792215	0.976536
140327551	2.408170
15648705	1.420675
48977229	1.327695
123129633	0.782198
125113727	1.131261
43784423	0.766665
41002740	0.783829
11392256	1.142690
42502252	1.119933
45839006	1.326334
121376828	1.365874
39301078	0.917771
140155032	1.129465
3025984	1.464816
69438704	0.633057
140705824	1.228816
50389225	0.875813
140495304	2.067711
56306669	0.733129
85063029	1.153724
34247038	0.683635
93684996	0.868152
40402100	1.106533
74361785	1.288759
117501764	0.747828
140155006	0.950909
95535427	0.609955
32014335	0.892398
54011212	1.747783
83645143	1.053094
44254666	0.906360
35298549	0.754139
32144928	0.846275
56306665	0.800076
2960812	0.652397
44865195	0.854776
44185471	0.840769
45342230	0.763630
128308524	2.098900
92336279	0.833444
92336311	0.654585
42893518	1.189113
2895239	1.955592
117707796	0.774217
40997846	2.184028
115377855	0.825805
43076757	0.676343
3049283	0.616573
90057554	0.965862
48013240	1.311170
5148258	1.180462
140506384	0.942062
39272297	0.667719
72654594	1.314075
2816238	1.091435
39390633	0.984240
75930779	0.786130
44389040	1.668088
43404352	0.690373
43934295	0.848515
14188485	1.032171
37208453	1.494160
16149297	1.376918
30569832	0.941081
124028718	1.222063
47093174	0.644267
139567766	1.094717
45070243	1.969696
33175866	0.709084
45288139	1.005761
140507547	1.884037
43022176	0.661773
18688865	0.772884
140566033	1.607156
140155056	1.079232
2895478	1.487463
139683745	1.898230
117859800	1.249066
15477300	0.679286
115537924	1.112370
102185029	0.672651
44674343	0.601816
135202572	0.733945
56306667	0.681859
14840782	0.660541
116565806	0.675819
62388003	0.959747
2909844	2.210440
45401823	1.339536
141322227	1.348104
39289528	0.990088
117435293	1.496571
31407768	0.690050
30571594	0.893783
140495742	1.524963
15336243	0.704790
137273617	1.395809
40971276	0.770151
41037771	0.673679
115856288	1.866926
nan	2.548566
45404116	0.946192
134029160	1.170594
30133924	0.752508
141225243	0.950619
81857383	0.982979
57992111	1.756607
5709175	0.663262
140951444	0.678741
141630623	1.179441
134029163	2.189858
137807340	0.730034
2820904	0.844251
116765595	1.278986
132858693	1.197725
6399465	0.733542
32607450	0.955095
122560497	1.554556
141322228	1.421772
140506720	1.211243
66455336	0.824602
91338374	1.952400
140669153	0.935966
14307713	1.112929
47067960	1.034924
46679637	0.876514
26096177	0.966702
45126520	1.960892
41037749	1.068997
45070917	1.630908
2987500	1.399459
64194446	0.849741
2925758	1.225813
112617454	1.064605
47521134	0.888293
42054904	0.801452
34616105	1.246350
44966714	0.670694
36280957	0.799004
53668760	0.848109
18819529	0.786373
867242	0.903068
45030265	1.236033
96725523	0.605865
44789242	1.464262
42292841	1.098681
141484258	0.692204
116830744	1.231358
140565456	0.811093
13873850	0.951033
83206798	0.759633
41810683	0.809217
66821013	1.015795
140899214	1.426321
140163501	1.051708
138328289	1.204048
32017498	0.990544
47751636	1.496715
32018169	0.821867
35147342	0.751410
138945831	0.709479
140699344	0.825720
88000571	1.010212
128927704	0.875822
2994886	0.882958
3028137	0.703053
44422370	0.765833
37356608	0.859188
6068997	1.024350
90845266	1.590161
40320630	0.676278
63890528	0.791296
44440924	1.320509
98598834	0.669086
140497405	0.997452
1719784	1.004120
139391859	0.904219
115541345	1.775847
73565211	1.415082
37256181	0.601604
1939145	1.096302
14352874	1.607927
51593752	1.346011
26146055	1.291918
140978635	1.402674
45799912	0.945415
140023635	0.870606
46129431	0.940111
135004802	1.013783
122733301	1.087144
42746177	0.748774
90162406	0.798211
15155937	0.612942
45255322	0.612358
15517309	1.432637
135604196	0.675500
46713709	0.706383
31933075	0.835567
44439174	1.056196
2826360	1.992076
140506856	0.700066
46102573	0.620370
115537913	1.774827
45456413	0.664760
140489406	0.982048
88699172	0.638523
43998881	0.694677
87484204	1.060903
27821045	0.880041
136985560	1.099005
66570680	0.996821
103187726	1.125376
64512995	0.947360
33979771	1.298399
39184634	1.183626
15663337	1.237699
140946854	0.879571
141331101	1.229989
42416940	0.965294
62295079	0.708881
116765566	1.326788
28846147	0.829289
46184825	0.651808
94636012	1.043231
126062691	0.641707
37300694	0.610949
104264697	0.927130
2856364	2.022710
91534724	1.307061
127550856	1.656242
31155799	0.730194
84077522	0.634322
34910924	0.836696
115538067	1.007353
140944022	0.667929
37473683	2.081827
44253095	1.193312
29197467	0.882419
117294695	1.640996
2903884	1.530949
137271718	1.191932
140476010	1.631703
87300179	0.850177
13974437	1.078168
39289525	1.137461
46119778	0.724953
140897368	0.669270
37356616	0.757134
57612040	0.770770
41210596	0.978408
9202870	1.081353
34910866	1.566781
46447668	1.099551
138945825	1.083144
39145587	1.409432
42435991	0.995630
87500721	0.692387
87483556	0.971871
54109356	0.856258
8279871	2.013103
31415934	0.825841
130960073	1.141823
14348233	0.886095
139973560	0.929271
44584104	0.792185
2927575	1.103217
2980304	0.878368
36810839	0.674975
89077542	0.660813
72654572	0.780553
89077235	1.374473
31415929	0.847896
39034399	0.695447
70212296	1.115795
44973994	0.742433
3012164	1.087279
127658176	1.138281
117181175	0.900004
44243381	0.714480
42568081	0.787457
48063366	1.322372
94633773	0.812233
15793697	1.255373
93306382	0.693695
139240073	0.915716
140565750	2.067702
15574246	0.804191
42321900	0.948140
115537863	1.299123
61847000	0.838785
39301076	0.807220
45165143	0.983110
140156787	0.825926
44440925	0.795051
2857021	0.699693
37822491	0.848996
61749461	1.058742
41008489	1.333003
132035121	0.642137
38254363	0.717336
14610406	1.027584
3022642	1.529423
26214108	1.871526
125180952	0.859743
138273578	0.808595
117707809	0.884782
34247569	0.980209
33988224	0.945257
128123931	0.785811
140495192	1.064101
24268642	0.831280
127658524	0.972319
127658532	0.710502
127658096	0.954239
18822490	1.325060
114271568	2.001080
42462681	0.783755
42463251	1.407134
37281851	0.965422
69438691	0.647386
20422380	0.649455
2802971	1.398990
140829988	1.053882
139812545	0.726777
75072589	1.597887
68143969	1.049707
48041215	0.767009
140840504	0.973942
140495652	0.880354
119338576	1.116062
35333463	1.245209
15574848	1.886761
29145009	1.209651
126057608	0.667411
43564382	2.022942
44408321	1.304591
117435874	0.638922
117708147	0.791259
97071143	1.134241
78337467	1.191477
99666935	1.292894
141668298	1.101467
117181804	1.392918
56064170	1.230527
70073651	0.816489
47836214	1.014457
3026435	0.679077
70073658	2.135461
121090421	1.406312
21631549	1.122161
2816256	1.185108
87933185	0.744704
72654593	1.104665
140476048	1.929033
141707189	2.789402
5722791	0.876530
87956531	0.708629
118774611	0.751881
62440104	1.020375
47849025	0.934967
47363508	0.746481
62440105	1.395481
43644918	1.138423
13985017	0.603051
32619423	1.312449
132356613	1.349422
115856210	1.586784
72654585	0.669615
9184858	1.612552
39324158	0.762215
26213859	0.619653
19261485	1.000161
123126846	0.884956
5718012	0.939160
89262614	1.159585
4186027	1.923770
115084724	1.141024
17438604	0.817248
16311697	1.159714
79232011	1.666568
867787	0.666695
73074554	0.892965
140495262	1.903780
37619318	0.913156
15455226	0.923784
87933234	0.757144
40320517	1.254980
88972662	0.743723
45254512	1.286994
13849456	0.754244
78168606	0.644995
9198058	1.732908
33790317	1.485402
140798729	1.073354
2995281	0.657774
4030385	0.672833
44200534	0.628223
61101332	0.757440
42340648	0.791785
44253153	0.943060
121084571	0.844950
125113737	0.936096
131933753	0.900268
45510494	1.327840
42399631	1.573656
90670423	1.116108
42399640	1.335890
41501676	0.627255
21383166	1.151811
42435599	0.655082
57992650	1.095537
48032817	0.696042
115377807	0.656704
85316901	0.686215
45332013	0.629446
14577625	1.847429
140327553	2.124573
31933006	1.027965
64513620	1.160035
45235744	0.673564
97091876	0.869222
47833907	0.792852
47859140	0.788483
96717474	1.967693
54424577	0.711210
1939045	0.769310
41606126	0.709216
128408631	0.747075
139970790	0.681408
141291293	1.316015
41855965	1.342821
43753213	0.720912
35301972	0.613115
40970859	0.786999
42541072	0.885446
2861210	0.774955
39939240	0.772069
40404810	0.617532
44917198	0.817127
58103944	0.994914
27636555	1.128140
42649882	1.228488
116255438	1.228988
87299339	0.773807
15536305	1.133115
116766138	1.080055
37619534	0.700857
2990576	0.890701
34455710	1.178299
47877820	0.687018
44938735	0.644807
78094603	1.002918
48041330	0.857285
140738509	0.721723
51369372	0.832500
45243942	0.734948
91338375	0.819702
30567220	1.026374
61711638	0.837510
45244030	1.241926
72654582	0.618546
51868648	0.608005
115538981	1.170303
140316315	1.788712
35429648	1.110344
141307313	1.977902
140587776	1.124234
46135094	1.421175
136982462	0.681528
33718838	1.007123
139970095	1.802811
140745615	1.949526
80671151	0.920874
5141311	0.730588
140780741	1.197874
46362098	0.732534
41813445	0.892424
90912197	1.156428
30625479	0.671310
139218287	1.499698
65239000	0.786414
37446398	0.664816
115314367	0.801810
33914893	0.713754
136826411	0.852487
45228817	0.728201
44581256	0.929865
38489959	0.758639
140172791	1.280865
32046924	0.935143
115378253	0.855059
4251068	1.414737
140509891	0.637390
116765298	1.544638
25427563	0.606845
70771359	1.077803
134022308	1.154568
140506302	1.077774
28737657	0.745912
36476727	1.427179
9185053	2.205083
53774139	1.408919
2843341	1.063358
20946296	0.952989
14688992	0.728605
140713282	0.773724
115854295	0.665684
32669749	1.134637
65249053	1.046999
48090493	0.769427
38777161	0.685578
26001636	0.682908
34378902	0.887509
140178993	0.825162
16569709	0.639294
91328312	0.626685
35428681	0.639486
45291323	1.159306
122241423	0.860545
141021732	0.988281
78442747	1.078684
141021528	1.158890
140475151	0.701719
105566247	0.900317
140324595	0.654044
139652400	1.176663
39272452	0.791540
141115692	0.644717
68350698	1.024227
141580568	1.004019
120158705	0.637078
141021747	1.083958
139655429	0.985029
31406453	0.849845
29873025	0.928853
140701436	0.865089
39136863	1.010580
46378686	1.042796
40020063	0.819405
14254579	0.776829
90057745	0.859363
140023632	1.292882
139683755	0.653042
138530730	1.034180
47921944	0.689816
141225245	1.182450
137995915	1.980954
140755703	1.372556
88851073	1.437503
120966036	0.847077
20747251	0.814428
39981203	1.789348
16430800	1.079699
45161494	1.256131
42425019	1.089160
141291000	1.420932
114305931	0.688801
44440866	0.699949
2857261	0.870008
38505481	0.902666
139566125	0.649682
112613921	0.842321
44153115	1.367421
44172418	1.366859
126062764	0.610722
15460461	0.915338
140652486	0.719319
118774590	2.046758
140315569	1.343020
139711021	0.915174
87300140	0.951154
83641872	0.610835
119073126	0.688272
44066824	0.626917
51890389	0.893007
6399455	0.762612
47687086	0.610530
47618227	1.401784
56051287	1.520574
84741728	0.967690
2928833	0.784871
127449202	1.281265
126056806	0.999539
2995354	0.796080
1752636	1.018220
950048	0.834301
47602075	0.624116
91775351	1.208079
20861202	1.109373
72683079	1.277599
6121676	0.851523
90057095	0.807937
70373552	0.628484
2830226	1.436444
57338564	1.015332
44380864	0.647222
30651070	0.853482
15585851	0.717860
72016207	0.950735
2842786	1.173767
16435667	1.029270
11416168	1.068750
88669062	1.409927
116765288	0.615251
40336426	1.560890
43508727	1.160809
64510874	0.859193
117183609	0.619093
6372969	0.954843
42426373	1.537280
64508289	1.636491
85297468	0.702919
115314045	1.354515
141330876	0.928305
15442693	1.206800
115539084	0.697860
46558537	0.652116
45565047	0.966944
139722958	1.226183
134029166	1.750403
43520317	0.749531
104264734	1.337642
2869112	2.245942
88972348	0.670325
16333346	0.987716
124054838	1.169091
42419586	0.771007
30570146	0.740140
134029162	0.812333
2926434	1.052690
64484701	0.688310
48899858	0.754945
87483774	0.842722
140193046	0.796805
14067197	0.894242
47618225	1.446725
2938325	0.868979
140476696	1.859432
72051828	0.915741
140222726	1.113207
115533697	0.693128
17469115	1.452700
140506848	1.358963
35290946	0.834147
64195172	1.245423
139967569	1.053365
102982042	0.704261
44264725	1.063871
87956528	1.726547
44127735	1.616559
42492197	1.569116
42650331	1.017431
71559785	0.997694
117434827	0.780351
18689049	1.139568
140538837	0.848410
62388070	0.822943
3503712	0.702246
4143685	1.395606
140791848	0.840708
123133509	1.890270
40997441	0.672517
62440102	0.622612
47772544	0.735566
41967509	0.708863
2843174	1.206926
20900804	0.800414
115538176	0.622039
45376417	0.770937
2895475	0.821737
90057986	1.038851
15575083	0.745366
47361670	0.987612
43916856	0.949915
44902386	1.544222
115538976	0.662029
32181423	1.076411
35480474	0.634035
41037033	0.621777
84741662	1.208506
44290497	1.099667
39182710	0.703347
141298417	0.919390
38721101	1.609995
115541344	0.775775
3886329	0.621101
124302005	1.623773
2860940	2.219647
46499747	1.062245
115539080	1.840832
45342228	0.824648
127449065	1.233745
61120803	1.592820
55486656	0.784991
44402930	1.297117
87483614	0.781732
42221439	1.026556
43040167	1.296930
117859345	0.956547
115084774	0.776909
76667349	0.757620
140699041	0.759635
46680407	1.012257
34084202	0.720318
61153809	1.410843
3061732	0.829246
139574197	1.507934
115860677	0.718949
133631672	1.077303
103373597	0.604886
61755498	2.006230
139922230	1.430282
116765995	1.066196
45326959	0.603468
52372526	0.639881
44654094	0.774088
15141432	0.720621
9001433	1.142592
116765077	0.981763
42353594	1.645388
91775360	0.683107
122735393	0.694667
100141925	1.261395
140794772	1.432093
2870679	2.449554
3000569	0.810356
30625552	1.208463
2930930	1.912422
140590146	0.695979
2896975	0.827835
18667035	1.002170
45626283	0.671754
35322468	0.852613
61154125	1.051712
62295347	0.631482
33988893	1.246835
61452550	0.889347
39927050	0.945530
16335148	0.900946
39349515	0.900273
140794017	0.653074
88701631	1.713686
105566188	0.985447
37239102	0.968020
117051919	1.181714
83642034	1.667865
139270904	1.059636
45608173	0.642022
45260867	1.489209
63878531	0.871143
120966108	1.366252
40401285	0.684692
120158732	0.888357
34350087	0.641998
115854002	1.029568
116572489	0.635025
120982638	1.473298
33841035	0.842587
47877286	0.988829
43813808	0.841469
27656578	0.932951
140368133	0.783987
42463250	0.929868
135202345	1.565993
84590171	0.707269
115855959	0.936343
2908628	1.133423
47918932	1.287445
44844457	0.868205
115538006	1.539429
9202865	0.746748
103187655	0.908411
55874723	0.835127
139186950	1.889391
117181898	0.631880
46480587	1.413727
42605546	0.801335
64546870	1.324857
40952712	0.938081
74645403	0.688048
39224480	0.683133
19001311	0.858103
15408036	0.684214
114305897	0.753995
64219681	0.806261
41042582	1.395988
16309514	0.799040
13580337	1.372208
6052283	2.423481
47877916	0.673204
14840192	0.788987
18744284	0.929014
88686474	0.650619
70483708	0.943748
38238024	1.089761
42486194	0.715292
2932649	0.606371
43040271	0.734715
47929402	1.237011
26751538	0.846436
120599275	2.005899
140163830	0.806581
87934132	1.154013
44336759	0.953556
76774466	0.821943
45244281	2.125971
46197273	1.815615
2843385	0.905487
15958646	1.513733
15108580	1.222314
131933758	0.836726
127449133	1.031680
40940185	1.211871
116765370	0.729868
93316585	1.027701
44660095	1.170162
140979133	1.031823
42541538	0.667585
139105004	1.008696
19676097	0.785703
141272954	1.481762
64229659	1.031485
68718922	1.270851
141040641	1.189828
2904181	1.073475
56047557	1.150986
39981239	0.705332
17637917	0.634098
46128438	0.716369
39179303	1.461121
140439705	1.907236
81454802	0.798973
32165956	1.121012
120604795	0.681645
40342762	0.902746
50201004	1.257807
43814560	0.781948
40997611	0.750469
46383459	1.139487
38550877	0.645574
6097228	0.632214
47584400	0.787333
47394245	0.651200
3023733	0.797124
123877330	0.912469
41612823	0.930657
6574597	1.307564
30674167	0.961345
43020258	0.652517
118010864	0.702645
140646772	0.888259
141247420	1.009529
140645124	0.865845
39260260	1.044157
42399647	0.980473
17438190	0.902604
35676541	1.336769
32017482	0.847309
140841093	1.653782
141247425	1.116872
141668303	1.071429
35322937	0.835393
34205407	1.412733
15795566	0.864924
18424901	0.714150
45626275	0.621424
120603981	0.751082
8898801	1.704029
88779340	1.317770
140764155	0.608452
35359326	1.013081
2894294	0.950086
62292842	0.673072
140706316	0.685956
3002956	1.306973
141023404	1.044674
88779334	0.828323
45269604	0.722753
44337230	0.809074
31362153	0.978662
50703284	1.118644
40059534	1.000271
2947988	1.062202
141094768	0.698718
29017817	1.282356
140306520	0.706016
57992498	1.182873
44199724	1.158701
44857773	0.906503
93682051	1.600611
125180946	1.182951
141630617	1.066471
140507546	2.036457
3151350	0.717325
14572130	1.942206
140496940	1.454956
140766301	1.208220
40891136	0.784274
15656937	0.872352
139996712	0.722947
140554257	0.934060
134649990	1.406532
40211779	0.836821
18772136	0.865040
5716736	1.031847
140193037	1.251788
140172261	0.987004
87500717	0.717432
26400062	1.014727
3850538	0.616193
21383272	0.905192
115856038	1.185110
15531075	1.159024
139247156	0.999216
2977487	0.777097
17438429	0.632344
43812558	1.051926
41365315	0.646160
45454231	0.812966
33152614	0.699889
37822498	0.997015
88000576	1.144390
29021215	0.713036
140713064	0.831847
37937338	0.710335
113918554	0.604523
87933237	0.839567
2958875	0.608452
140163832	1.361765
83642330	0.675042
41844700	0.808895
140163727	1.265160
47877076	0.734049
11435020	0.642516
34389146	1.050309
141094773	0.939129
2990550	0.874677
140163722	0.751327
140967865	1.381939
44658340	1.041722
97666582	1.006201
47753176	0.648702
87698358	0.818801
18883874	0.799981
39671458	0.917230
119615139	0.866166
48167796	1.463048
89077083	0.834282
140640699	1.626493
64002282	0.929439
32601765	1.594437
48040800	1.070836
24268522	0.750306
141090695	0.973924
140327564	1.165084
140979476	0.625327
140899209	0.711152
76123087	0.895925
47873326	0.668888
86909921	1.390470
47947746	0.623878
2826218	0.914326
33988890	1.265475
32017425	0.618150
141023600	0.791171
41763369	0.602584
75820004	0.618634
85825015	1.085693
29033995	0.740450
139270906	0.943270
119072469	0.610271
140047084	0.610096
16202745	0.636265
40338656	0.672801
33718509	0.750308
132652423	0.627792
141148225	0.860702
64400233	0.866223
140509922	0.869023
50312564	0.834216
140699399	0.781807
41027206	0.826013
5728662	1.111180
132354991	0.746042
91429792	0.837576
36840760	0.923814
62430061	0.711424
24268643	0.781434
21266086	0.917618
125784011	1.222075
62388378	1.174568
101814265	0.659270
115856137	0.643080
1752733	1.066118
88699185	1.135311
3026578	1.020413
140554256	1.104311
140146892	0.921405
3011018	0.877451
140042218	0.675996
33868431	0.777500
26431284	1.339227
39282773	0.670179
26189924	0.717749
71468104	1.724675
2920172	1.298816
45565046	0.644060
120966050	1.417797
44252610	1.785491
28946207	0.653977
140799580	0.998599
46999161	0.862294
32184697	1.213686
45069217	0.940047
43795951	1.278619
73148913	1.535594
44195795	0.683936
14091294	1.459594
27328137	2.189971
40158232	1.356840
37402701	0.696422
139244069	1.391371
93434597	2.426030
93434688	2.005652
126056812	1.332842
28963629	0.811469
2908729	0.672751
64401049	1.611281
53842147	1.089300
39144186	0.709150
100147986	0.845377
73764954	1.151621
135334824	0.813841
36736177	1.278784
40974285	1.173019
20960946	1.006945
69179222	0.802150
14580994	0.635720
47877799	0.647453
140583856	1.008389
3076152	0.682459
140681995	0.730011
75820051	0.607775
115534073	0.863936
78485882	1.347838
40891138	1.123140
140899212	0.777784
27318910	0.945542
132872381	1.258565
136982453	1.071644
47877772	0.684715
91328219	0.894073
63890451	0.841830
44635978	0.699540
15629184	0.837197
37821851	0.723455
140538432	0.857613
21088303	0.673421
139231165	0.877456
140791854	0.662447
140791841	0.864540
13857640	0.652742
14644548	0.656343
87413199	0.631576
140767313	0.876663
46309388	0.610269
41035196	0.944385
45064002	1.184419
2797802	0.617604
2904733	0.738613
66890118	0.921302
39502115	0.671512
46693882	2.081620
15029229	0.956522
52863678	2.124650
42523942	1.952173
3003871	0.987485
6102177	0.761445
116565786	0.990532
42000620	1.157382
2903327	1.004780
20940900	1.326084
13712033	0.956645
87501892	1.109554
45097479	0.700171
120966060	1.142755
3027827	0.708796
18991296	0.783379
45442920	0.644161
47827096	1.180767
43279662	0.800162
132035122	1.396179
39204625	0.704732
3036432	1.147508
141300619	1.521726
115933124	1.791213
31405171	0.878795
141040642	1.092345
40740715	1.029154
23699848	0.726864
2996681	0.690111
46145745	0.641151
46202813	0.642466
6597612	1.609561
2944982	1.133702
29844105	0.640107
42541520	1.201824
41798170	0.863139
64523552	1.001264
140559503	2.167899
42898967	1.532009
74212148	0.905373
140513568	0.789835
46704721	0.715886
44127733	0.923117
48046375	0.852750
39182955	0.845510
116766017	1.570857
115541450	0.699026
56068041	0.623011
32184631	1.285861
15515790	1.623016
132038882	1.673223
54831525	1.001473
44261644	0.672552
15366582	0.760226
103184547	1.375993
46694378	0.815379
43168195	0.932888
141529991	1.042394
64507929	0.600614
31933196	0.843581
36775887	1.839640
48900012	0.650323
40402069	0.991620
103554808	1.700484
35298487	2.139256
141103153	1.000847
2962854	1.209001
47057492	0.862016
46576306	0.838317
141307159	1.419756
867340	1.314512
104898233	0.736742
35051411	0.923701
113927390	0.864683
115541364	0.962281
68891207	0.642743
137405842	0.662663
141302709	0.629176
140790791	0.919272
17470457	0.748494
16377915	0.932556
88699171	1.425116
18607403	0.703621
36502062	1.071431
17450102	1.929242
2942018	1.114961
29906916	0.601379
140163355	1.443873
38210255	0.701987
40644066	0.919349
47241346	0.809194
42221432	0.967676
20900800	1.225371
44380785	1.473855
21439221	0.637031
19908061	1.049736
22485611	1.001260
119398849	1.174271
18688665	1.183709
140783889	0.999995
20900903	1.680539
140315440	1.537735
13899170	0.998269
43584212	0.833350
17198603	0.621121
78483407	0.999856
139105005	0.747090
141016543	0.835680
62440103	0.699530
2828804	0.700805
6117624	0.847447
14576563	0.641065
44066652	0.617334
123757132	1.091437
44127671	0.611321
3012996	1.209549
42893876	0.770692
29169918	0.600437
140705842	0.705399
120838107	0.756006
47577713	0.655170
30595686	1.868451
2869278	1.676388
136982451	1.123789
16513173	0.786841
17570655	0.965532
32184578	0.680585
2078040	0.624089
45005237	0.915684
37867378	0.600087
71623104	0.948964
6250011	1.091122
26771893	1.031771
61452697	1.132729
43755959	1.486411
29691673	0.681615
46112171	0.613081
38904361	0.689526
26240717	1.492264
15793367	1.593444
32173939	0.668556
42237067	0.655947
42886581	0.895896
47878294	1.069816
34246070	1.065131
18884170	0.705810
140147623	1.273154
43779817	1.919459
13713036	0.646267
2902972	1.223184
2842789	0.923500
3011236	1.138015
39294303	2.020692
5722780	0.788468
84103564	0.970078
134868993	0.712095
87484038	0.921671
104470895	0.761648
114271638	1.307596
139518552	1.697745
43969848	0.623816
117708329	0.848183
21265556	1.611985
47946153	1.058360
42221854	1.114542
35144803	0.622368
88972710	1.201802
39722342	0.880123
120159845	1.345961
83641839	0.756366
15177818	1.130275
32162894	0.873870
44564413	1.546088
115378134	0.750073
11445220	0.630710
63569727	0.718682
15143929	1.309254
44676015	1.181453
120966009	1.264526
140966389	1.277851
26003114	0.928836
44546657	1.240503
21279268	0.773201
34001006	1.067140
141331100	1.862263
104129676	0.966485
26952022	1.197694
47789672	0.735440
117182036	0.710609
2909202	0.823490
139315214	0.856181
15551893	0.655585
119398846	1.345279
140506604	0.819301
2915942	1.016004
141165632	0.836815
35384476	1.537508
48900177	0.707234
45268685	1.303864
47873284	0.669026
4533087	1.248771
8279865	1.331721
45243490	0.975637
89075064	0.954046
1780349	0.983856
3015012	0.984472
41008610	0.644991
15037888	0.890455
62427002	1.275692
2868398	0.908189
6399462	0.845776
16423443	0.705319
26324447	0.877308
48034000	1.495720
140155003	0.835907
119615188	1.206427
35481968	0.895069
45570384	1.709542
43076351	1.069215
34064592	0.712880
136826407	1.297499
139569994	0.815992
7786585	0.613849
29411845	0.962862
26781510	0.720403
20422535	1.309402
61317645	0.625941
2924791	0.911036
45324616	0.651143
70066382	0.651486
123878275	0.778309
127880056	0.743851
66171505	0.616642
2951640	0.978173
39134232	0.943431
135334825	1.223273
34797115	0.635428
141040516	1.127496
30585374	0.603774
2842717	1.794165
140955480	1.457881
18775888	0.780527
44253011	0.983544
16932172	0.938803
17569352	1.629977
72650478	0.652333
35332429	1.034314
39104604	0.875272
141273018	0.601230
3013508	0.794666
26796844	0.941559
140509892	0.627784
140920714	0.855782
45291443	0.693038
4559952	1.353511
45255468	0.808746
13988853	1.129838
20936448	1.404451
30570762	0.875637
140474819	0.793105
77691006	1.382368
2878324	1.576580
97222568	0.939834
42331222	0.787843
2814956	2.189117
11389396	1.374537
26431241	0.946189
85239345	1.383717
119973194	0.885797
46384862	1.022473
39374877	0.836838
45165551	0.894320
141527447	1.035374
101403732	0.800304
42516282	1.146225
47542945	1.275190
53440571	0.778761
3043856	0.609223
15532070	1.119990
140759077	0.784834
64735722	1.346739
79409154	2.048899
18834102	0.845509
41042535	0.892677
45278357	1.020272
140793939	0.703433
38579087	0.689161
21631066	0.881164
35525932	0.710031
16346216	1.277805
47872033	0.743329
44625115	0.845641
115856163	1.053262
100670129	0.658960
139432016	0.972945
139276216	0.719424
87483486	1.000850
98343377	0.788393
141023211	0.751597
89597456	1.304750
120966067	1.312573
140305572	1.924949
141023209	0.946828
72271719	0.672687
3896641	0.629296
39289983	0.647347
15527789	0.705357
16346390	0.985621
137413146	0.653603
15159667	0.712608
64510375	0.890043
44262201	0.765553
55874628	0.631069
140091891	2.187248
16201400	1.578815
140496100	1.208388
4633172	0.882751
3008280	0.639081
11430945	1.095059

2965101	1.001677
64195176	1.571292
42221414	1.804996
33990457	0.879664
139218637	0.962030
64438207	1.190539
95227195	0.643711
93306756	0.697059
90323642	0.974599
47756119	0.722661
47501776	1.157788
44857775	0.719969
138327722	2.398575
62388307	0.986068
84077435	0.631267
4203816	1.443529
31148645	2.189646
4142940	0.758540
140193058	0.850779
3004265	1.777510
6598312	0.913431
43835734	0.893237
140538537	1.793686
115539032	0.928330
35193924	1.113079
140189307	1.165949
34180052	1.397278
64551322	0.807019
44589066	0.718125
26689783	0.724347
71133802	1.522020
115854316	1.491717
1846568	0.604116
140723361	1.167110
15557896	0.698958
45367917	0.716328
42785207	1.251399
29880705	0.985735
140509825	1.083571
64529706	0.728910
45244847	0.608490
141293146	0.842783
35468523	1.533107
141293156	0.913022
26324850	1.683258
42785167	0.949692
15714875	1.041606
18681207	0.709828
84741624	1.783936
30653227	0.636675
45099481	0.692384
140076264	1.083934
140559456	1.706146
28846152	1.239531
37438053	1.389474
88790408	0.780738
18260543	0.627038
21266184	1.135610
138705557	0.838107
18767646	1.574172
141115693	0.797154
4559931	1.022362
31356559	1.287311
2868987	1.051090
42368000	0.685789
35344651	0.994908
134029159	0.820343
40414900	1.107188
1787228	1.214477
66254976	1.322773
140507545	0.660222
100146511	0.707328
46699459	0.981746
84590114	0.870729
30133958	1.575251
139869612	1.043352
32820575	0.725172
88973266	0.743182
62428045	0.621725
44290204	0.906412
44389222	1.457461
140397008	1.459220
30531352	0.601111
45188129	0.711667
15751693	0.675326
57992160	0.676537
116765127	1.148224
23623141	0.932093
30583856	1.330914
103376593	0.922829
118013587	1.089367
44583889	0.960433
140583119	0.765360
104264768	0.615402
117181972	2.010036
35374534	1.858410
73028667	0.697177
40414848	0.802488
11309901	0.744678
127442324	0.851416
42555267	0.767974
103187729	0.802161
139356262	1.055511
120838062	1.062475
34332678	1.058317
15177737	1.100218
4502352	0.604608
6274414	0.612693
63890953	0.968972
3015014	1.157941
2903814	0.630584
2903816	1.391941
2908627	1.705405
14365427	0.613420
4502799	0.668501
139583730	1.074150
2958431	1.005545
44267746	0.850963
87483685	1.043052
4172296	1.250170
32464658	0.831997
140156330	1.186527
8234711	0.904731
30611657	1.757446
116086615	0.690168
89597436	1.500735
83645127	0.908850
2991201	1.220846
64514573	1.876172
30628720	0.697507
115856805	1.333981
115314085	0.610109
139271248	1.109951
116086532	1.366085
32412267	1.251522
30595694	0.997341
98640299	1.031164
16732986	1.001036
16513345	0.874727
93309624	0.730708
47763030	0.804259
87698357	1.682036
44127668	1.141193
30674222	0.706394
120838008	0.673729
141039405	0.852613
71244421	0.687824
4670810	1.893381
8931675	1.013828
76123986	0.830266
42116488	1.470290
29015896	1.050218
16229448	0.835823
45025824	0.682308
1662693	1.151912
44422349	0.814502
117181794	1.261765
64483599	0.644630
141580569	1.435461
35298560	1.657237
16346252	1.121164
32160054	0.920991
45344116	0.700436
140701568	0.972674
3001123	1.338755
3643452	0.928079
32018146	1.690046
35469866	1.201461
45242356	0.954620
117014601	1.200129
6593106	0.676600
84589949	1.018980
62295148	0.755510
140506006	0.650591
11428437	0.802880
11596517	0.755873
21111462	0.678285
32060825	0.804878
43621762	1.264139
138530711	0.939884
91775047	1.362479
91087882	1.062369
45242354	0.613233
139355303	0.731063
61669796	2.340997
55874505	1.220859
3203053	0.707718
47511433	1.831531
140172799	1.220233
91774758	2.058366
64514475	1.448821
116302020	1.409745
44902376	0.715611
127670833	1.286090
84078046	0.665928
120838001	1.721464
87483922	0.938785
45244823	1.194983
1754079	1.250826
26244626	1.097643
4560280	1.203847
127550866	1.908837
32017534	0.712303
139528376	0.743713
14249363	0.847007
105013717	0.729786
27655930	1.504002
17541521	0.902001
2985018	1.253533
44020616	1.469323
1577997	0.637432
32901672	0.621138
45261169	0.807304
2944412	1.575810
42398180	0.788484
128926644	1.417381
123126652	1.443533
44807831	0.631138
73221154	0.608586
46037064	1.151686
115860702	1.679678
139566749	0.858390
140315652	1.094672
6399432	2.114093
4605609	0.735337
105818155	1.548853
31178087	0.781217
3015182	0.616371
42781725	0.782266
89077176	1.024446
88699170	1.180780
30595702	0.815493
120604783	1.460227
64396431	0.937728
22840702	0.640273
140937358	0.756862
105818992	0.796852
27636534	0.782043
71560224	0.811587
35311663	0.881149
43809866	0.739439
88779513	0.979291
48024592	1.426247
138942796	1.226380
16431550	1.708820
62294973	0.644499
15513190	1.690739
15957789	0.783413
23050980	0.632458
5687315	0.703797
44879976	0.608903
140506077	0.862922
13835304	0.866105
95669979	1.179145
50253191	0.644777
44583886	0.721161
46481238	1.058425
6399357	0.630955
140975345	0.753905
79409155	1.976554
139879316	0.627876
21383160	0.815910
64495616	0.611815
29055660	0.721662
31397698	1.671402
100527286	0.896950
6145796	0.617291
23623123	1.373617
35138790	1.378115
140315370	0.713413
119398855	0.749396
105332957	0.640743
13907381	0.738350
15232910	1.149354
45401805	0.870551
45088082	0.757922
79809842	0.824166
25610952	0.838599
1775182	0.710146
0	1.501954
54193482	1.085632
123309655	1.369411
39927456	0.709483
2980303	0.700956
35147127	1.570542
6609307	1.326630
48077859	0.603670
114305668	0.609904
46299344	0.782987
31979623	1.255723
86989377	1.162447
14636664	1.055390
115860707	0.765893
29837264	0.738964
120838503	1.235229
39292441	1.037258
46498806	0.712656
42516503	1.744264
2896829	2.165229
47877708	1.073636
13950326	0.919431
45773503	0.810869
26923511	1.619159
63305611	1.196085
91775041	0.996394
44441422	1.265196
40997847	1.579438
103554674	0.842825
35146846	1.164271
44261690	0.676146
140966320	0.829337
43279752	0.864681
2984513	1.383886
44200382	0.654335
37822010	0.780820
84470247	0.730060
140156809	0.767009
77500220	1.108385
22485249	2.309901
45236254	1.049165
140979482	0.660358
115699511	1.043589
8187258	1.157272
8989525	1.353725
7979091	1.725582
115696606	0.838957
45273287	0.703359
15771347	1.299739
37066102	0.636732
130962720	0.637961
139104771	1.181706
3839154	1.332195
98111873	0.826606
27470577	0.762048
54562348	1.492322
46786720	0.653760
16433190	1.216847
140783907	1.677868
33046960	0.930283
46360995	0.748591
42549478	1.176705
47826793	0.642515
140171878	1.147251
140538846	1.239947
141038255	2.108311
90747106	0.927616
116765894	0.760961
118332672	1.036675
84741665	0.939965
11588912	1.714799
11377265	1.371202
117707800	0.844895
32184684	2.092331
65264116	1.501446
61103066	1.482519
92198658	0.641004
47878404	0.989839
57597127	1.270823
45367943	1.146962
127547462	0.850121
134568334	1.363927
47918759	0.847908
140880146	0.619480
48983266	0.863837
141605495	1.255770
71887344	1.012235
68718923	1.263797
131648032	0.860040
18693909	0.896954
34908355	0.879753
121623989	0.777736
139984508	1.546644
63890571	0.820110
21266009	0.707478
32184591	1.581599
9064221	1.587764
39233917	1.215701
115537899	1.047252
45409755	0.713934
117708065	1.065143
141023539	0.746240
51913985	0.790889
140091892	1.680052
41054148	0.685797
74104570	1.070200
15204618	1.449334
141293444	1.130471
130168259	0.885984
139658806	0.637324
140738558	0.889097
95848079	0.727003
4627451	1.209163
116765942	1.426485
64484499	1.016174
42609593	1.787046
37463589	1.041823
47904882	0.938569
3028768	1.733161
17827554	1.027401
139458453	0.872594
48041251	1.489918
85069948	0.885748
78479660	0.931677
15168156	1.047919
140509824	1.087240
128122775	1.476223
91961251	1.043412
4627242	1.043989
13863035	1.285695
43300483	0.618300
18741398	2.108003
42785916	0.859535
21471772	0.716604
140784417	0.605515
45077292	0.709855
26883804	1.234526
64002285	0.689758
140830228	1.291080
2891573	0.970832
139432522	0.606153
76123975	1.314462
37438052	1.064248
96600859	0.673054
42785228	0.849801
63760296	0.965024
48074754	0.698147
33841079	0.823696
35067262	0.842400
16377880	0.966454
17470075	0.606415
47753294	0.861423
132392556	1.099219
71392407	1.118090
40891106	1.788849
134030443	1.168326
45401829	0.610504
90057655	1.516602
139528215	0.618167
40997348	0.757653
2912483	0.974246
47918876	1.318198
136826278	0.694411
140560157	0.930948
40938949	0.650064
56048965	0.944581
1939232	0.716025
38836685	0.687183
26224409	0.871449
87299383	1.416051
47878310	0.747668
66255210	0.926500
138531862	0.913566
45189652	0.962129
3029154	1.059320
85746242	1.103812
139567194	0.723420
108225916	0.897787
17452452	1.225554
45359028	1.174984
85824973	0.666240
2837331	0.845466
4176529	1.524023
42445609	0.637292
45257434	1.118505
87618862	0.718775
47707740	0.760852
45246078	0.726571
41000519	0.681851
92335275	0.740915
47903434	1.202460
44477541	1.430968
141529992	1.133524
16512301	0.891416
42649870	0.893039
47795022	0.996859
31162406	0.907056
44169476	1.773562
43812564	0.825087
64483848	1.470960
15944147	1.031029
40931241	1.681402
139469145	0.646234
18772281	0.994805
6036346	2.008843
139429253	0.804893
134981587	1.117425
81481266	1.089745
97858003	0.805769
132138894	0.781382
16801162	0.760220
62294984	1.381629
56306671	0.697943
115377602	0.969268
3004343	0.658469
113539524	1.465059
41008611	1.090064
139683756	1.699176
118777356	0.871151
41496931	1.249523
44512685	0.771619
42785584	0.745591
3791985	0.683043
140889585	1.088000
141074963	0.980286
118332675	1.161949
117708240	0.604755
28846268	0.956929
42785141	1.128069
137353330	0.760181
140830384	1.191727
46155796	0.970987
35384473	1.533092
15493824	0.986047
43280178	0.632009
90911459	0.607274
137273470	0.611205
13657838	0.646334
115536917	1.571891
2908288	1.186973
6996553	1.148193
118332674	1.142158
43937222	0.694721
62292571	1.110277
141366551	0.858148
42030602	0.789312
118332673	0.868944
139494520	0.722413
45306181	0.713369
35454024	0.817442
867045	1.009426
27328154	0.661642
45372156	0.747716
17849371	1.099441
104283125	0.674850
14284927	0.725406
82851719	1.265710
45050256	1.229829
39145533	0.613704
15543777	1.201553
51914338	0.651508
91251685	1.144503
41225365	0.881291
11415774	1.121775
43972793	0.734364
42463244	0.776556
114271567	1.438270
140369030	1.839044
46102592	0.806773
48040757	0.882376
88699181	1.688176
2868894	1.037086
81448500	0.793780
26213827	2.334044
115854043	0.863354
39698820	1.259080
13398766	1.066725
23487024	0.679368
41362661	0.895148
88699182	1.404358
88699177	0.741394
140506918	0.608142
42412723	0.618706
2986705	1.432863
88699183	1.347004
138011848	0.774588
138706429	0.989796
2898924	1.165826
45407755	0.818597
88779550	0.755385
26324999	0.707892
35359400	0.762314
11301906	0.986978
140193002	0.771500
141631474	0.811907
92439852	1.088097
127880908	0.764171
14588838	0.600222
45409773	0.878466
139276217	0.653213
116252373	1.438373
116252511	1.431003
29273354	0.650119
140315204	1.479337
5989701	0.880860
141093853	0.863038
137924137	1.022106
34391598	0.886033
45351932	1.513127
46680285	0.924580
6120147	1.282089
17574386	0.730272
43873033	0.821956
39145489	0.632291
43950570	1.576598
45268635	1.470339
1750842	2.115379
42437703	1.222152
39097099	0.604855
34346133	0.880139
139973317	0.965804
32145121	0.684407
32276301	1.161143
91774984	0.694297
141286583	0.977873
33793496	1.134647
137995137	0.768935
90057509	1.008828
65289180	0.893827
45288647	0.903440
43724496	0.680347
45243678	0.788218
37026438	0.681416
141293141	0.787298
135202484	0.715061
30653218	1.111810
120158706	0.894916
9197897	1.333871
116765157	0.869442
7788390	1.195529
32964238	1.189612
46512773	0.870839
2904136	1.207194
47873328	1.156924
45244413	1.141828
894055	0.835420
127442539	1.633684
73890438	0.610123
6244838	1.126985
45557891	0.729395
136589112	0.642086
125589504	0.868876
40669522	1.020095
11379494	1.371366
140315591	0.695578
71882477	0.602564
39575133	1.424533
6367728	0.757685
114305671	0.848192
140919421	1.022055
26283476	0.767313
44521484	0.642720
42282419	0.691066
39223987	0.911142
85238212	1.145546
14379664	0.616459
45047317	0.930436
141377266	1.444499
1750675	0.841895
35428704	0.956837
2894822	0.882546
893213	0.858931
45287467	1.301811
47838379	1.271925
18773227	1.024328
1538933	1.391706
44678997	0.800883
56051247	0.751614
138067491	0.825859
141305499	1.096126
14825158	0.706243
40972987	1.036958
45580909	0.690332
61755501	1.090851
33812966	0.638457
47832524	0.814909
85162792	1.039910
33718576	0.906158
18884118	0.745796
882973	1.151100
40735419	1.127853
62292582	1.275652
36414237	0.722278
29021079	0.804493
33793954	1.268715
64404874	1.012922
140559442	1.181103
44160695	0.750821
48063392	0.621794
3004802	0.932800
32207037	1.212324
40208323	1.371948
29488016	1.428830
18537259	0.910250
116253114	0.720125
56063126	0.911277
73004930	1.152563
8931674	1.625970
46036634	1.739046
64472591	0.823792
35384447	0.830474
20960977	1.451347
126056843	1.305186
105359834	0.990893
88973499	0.677143
98608179	1.166750
39094781	0.908326
16402616	0.949182
14259931	1.163346
15248299	0.618995
62179424	1.082980
44420617	0.626197
37262006	1.141604
104264668	0.686389
127550861	0.722677
36773196	2.157860
21264207	0.795622
52202013	1.174809
15141218	1.246125
1752180	1.422919
40891408	1.155917
103346657	0.846069
33868386	0.961816
43982718	0.670875
141632438	1.604665
15144165	0.800625
62427243	0.745197
90656657	0.660506
47822159	1.850859
89077527	0.645579
45588037	0.669903
42546389	0.797916
43008994	0.990416
139392599	1.089884
15093009	1.138184
48046473	0.799937
44420655	1.094635
139105629	0.615459
54562060	0.748126
42436366	1.536170
140799571	1.022833
28735543	0.713681
42325867	1.317199
132858726	0.929559
44626136	0.727592
28852374	0.656803
15543555	0.773456
31388364	1.519146
45318468	0.646911
2918997	1.753507
94823168	1.563969
37905439	0.996826
27328329	0.866583
39554559	0.984447
2953395	1.693684
62388988	1.128718
15245774	1.387435
38990885	1.132598
91775214	1.052339
64190078	1.065627
91483867	0.754949
18819838	0.843658
44013970	0.758886
2954651	0.713800
41036562	0.794177
30986390	0.619827
93434929	1.364822
62389488	0.868041
30555004	0.706223
26141866	1.416826
33075005	1.108408
6115846	1.109797
120966053	1.123057
140396716	0.748941
47779343	1.111029
46918125	0.744428
139181854	0.685425
44261700	1.062842
44184349	0.941985
32017904	1.818299
18785512	2.102197
64514435	0.848794
115860658	1.837569
39182514	1.026927
14576620	1.471729
16501984	0.676683
91774776	1.110906
44561755	1.029514
40896024	1.182403
46173458	0.874846
120835465	0.878095
139721526	1.280638
40889550	0.666337
32069276	1.317765
38883336	0.844125
40889555	0.914605
37478594	0.746183
79276893	1.322095
44284748	0.796296
14120774	0.857266
90072106	1.351394
139105437	0.738085
139185719	1.269764
42000642	0.794454
44774803	0.614753
2790200	0.959965
73197557	0.953019
46032543	0.736558
14642763	1.657529
37871207	1.101016
30595795	0.796484
18744283	1.731045
79424298	1.203686
48013047	0.869764
3032641	1.425882
141307323	0.749998
15160926	0.694228
5724822	0.760495
42670560	0.646321
88973589	2.030554
78168609	0.711165
5724814	0.604742
70073656	1.739066
14305802	1.288898
103373649	0.849785
82552910	0.899859
140495238	1.571737
64483887	0.669733
116577656	0.750347
2943662	0.838292
42541220	0.893475
39262416	1.475221
83641782	0.686489
44865194	1.085638
87300108	0.761912
121084575	1.024328
43554874	0.608662
140794234	0.760607
39987599	0.757382
116765513	1.001074
2031848	0.911415
40334882	0.674117
18904710	0.711222
1751329	1.094374
140609439	0.632290
6249982	1.474823
36868748	0.846711
35415561	0.871537
18061817	0.779555
27774468	0.662472
2944237	0.877431
19881506	1.046927
39986474	1.247474
140815638	1.187569
140315209	0.811085
44624854	1.623551
16307287	1.139098
18679297	0.972510
30583942	0.870103
29124502	0.677652
19324210	0.865753
43644923	0.763400
29411739	0.642702
14226111	0.683510
45165114	0.831406
92032760	1.473144
14347041	0.893233
83644143	0.865348
39215797	1.881475
44953409	1.293600
140505996	1.150572
140474768	1.171064
30982840	0.682517
42614260	0.604904
6335168	1.183771
118775140	1.846462
44242984	0.734817
18693960	0.946296
29024119	1.150403
44470265	1.130411
33855281	1.098710
44243474	0.832232
9184860	1.120437
36230482	0.659287
3097308	1.697839
98598458	0.706715
140583769	1.299456
71965767	0.665296
77499862	0.770342
32650704	0.903097
33867317	0.612726
25604628	0.765651
140327561	1.129029
46662468	0.917470
2895063	0.981366
52698970	1.385619
64477439	0.953523
141686199	0.694474
44854334	0.934517
6356985	0.846007
44450633	0.740208
90057850	1.212110
140475455	0.603124
50634374	0.783376
2851475	0.738753
82837669	0.637511
68719809	1.033114
140919401	0.797404
46298464	0.635467
140565544	0.923503
34355342	0.644492
43020804	1.280114
63890582	1.883882
100141732	0.929453
115860663	1.263474
75354007	1.112632
73764965	0.701311
41443218	0.839863
11462985	0.973558
15026074	0.964612
141527444	0.937211
36239323	0.964398
118776745	0.716287
88699161	1.318536
47079350	1.634789
6329824	0.673835
50387923	1.070218
45255491	1.081579
30702022	0.630566
140729237	0.871504
64508712	0.678146
44315041	0.942989
33982305	1.659831
87300146	0.672843
22237470	1.065983
87951654	1.238494
48041263	1.163417
42762561	0.993667
45287911	1.464725
30407652	0.821947
91483944	0.607778
16801015	1.092574
41094697	1.430960
44084621	0.615038
42426354	0.967364
69086292	1.139662
15958349	0.726618
39158	0.970220
141501450	1.405604
28891449	1.308940
56516166	0.842292
39976213	1.078757
30085053	0.920974
28878491	0.606466
18455638	0.694334
62244148	1.058381
44015604	0.958906
64511574	0.987864
6399466	0.774638
92716125	0.618197
46119876	0.631366
44389118	1.231477
2934959	1.040961
118332691	0.625271
39294381	1.007207
90174993	1.109386
140705362	1.682250
92419125	0.771806
46918163	0.750896
141668328	1.067754
141022290	1.952147
64976421	0.625797
45327409	1.121608
72271713	1.268127
89597440	1.234662
141225226	0.635955
88699169	1.601487
71965519	0.633567
38188332	0.626447
2998303	1.251278
30508436	0.685994
32306835	1.735732
35327697	1.059438
41082785	0.853896
5581264	1.250187
6102129	0.792437
44135947	1.324656
46179176	1.092529
1575918	0.618717
115395337	0.640178
3034742	0.782350
15619281	1.042190
39281995	0.832019
41083228	1.089669
44161540	0.833501
44243351	0.988676
6659820	1.285756
44187156	0.669466
30531855	0.623611
47929360	0.708656
16563642	0.820943
140146891	0.648847
27821051	0.723784
113552439	0.655569
2799183	0.601949
140723164	0.648151
5740080	0.836303
15231608	1.724122
41080732	0.805274
45328675	1.537768
141632144	1.125393
127306575	1.530533
81482273	1.108886
33789920	0.917843
2902370	0.821891
46369065	0.654975
25713416	1.401116
44336487	0.840250
46104606	0.701465
139392603	1.111593
140023637	0.664204
16896534	0.769320
44381357	0.768766
41000516	1.093243
17631326	0.771857
119875180	0.662580
85283863	1.309448
17742099	0.756308
93045491	0.707082
4145371	0.802880
141619009	0.628910
140155242	0.764606
140583619	0.808226
38676937	0.660353
44264189	0.628965
140583814	0.726285
44261028	0.806843
115538741	0.830165
27538383	0.689852
39413276	1.111310
30595633	1.082415
137922645	0.903535
24752693	0.686097
44389277	1.868271
77901359	0.883161
2980100	0.971491
117181533	0.796165
30445392	0.698078
2973882	1.283288
137995882	0.613226
43439852	1.087669
2860734	0.757919
114305768	0.978171
50392324	0.854055
141331187	0.933622
140349529	0.955627
31960941	0.653141
44290429	0.636747
91252159	0.676100
86993719	0.965135
32017613	0.981944
141300642	0.767992
13866626	0.839897
47929517	1.199474
82851965	0.877161
88973058	1.045476
44590014	1.030438
140306507	0.798965
115378277	1.308968
104131815	1.451785
135229580	0.720567
29674973	1.257270
91534718	1.655902
138009638	0.875470
140627170	0.618681
140368132	0.666012
42374033	1.012428
44336891	1.489246
41587156	0.898303
105817912	1.945507
117181705	0.846960
128308536	1.054457
45519662	0.614252
33790247	0.831844
17572621	0.621667
139368882	0.916969
30625410	1.609667
30446819	0.911392
2955979	0.730004
45230365	1.515755
25781910	0.738352
73202169	0.677895
104270225	0.627240
14426062	1.073964
107202372	0.651084
37437283	0.743245
45244770	0.850116
133862318	0.874458
68146899	0.855660
46179356	0.712203
24415871	0.648781
61555615	0.859329
139574269	0.729483
132352291	0.689339
83200267	0.965895
4542864	0.656756
3023552	0.662600
41551758	0.717192
47092940	1.242798
35298515	1.916234
105816869	0.722168
27655856	0.621975
101402062	1.241956
106760727	0.720617
39176242	0.737356
15612349	1.066080
34945248	0.644625
43011272	0.869537
15407905	0.771077
75938786	1.202752
14592250	1.079483
134641183	0.703883
140843260	0.850693
45622638	0.759957
141038949	0.929784
30571353	1.050741
2996204	0.709885
64495575	0.717504
11393658	1.207703
98621407	0.789581
3851438	0.611672
30595648	0.977579
123133510	1.133568
18607404	0.735242
2955092	0.729992
130972157	0.942527
44300326	0.652515
28725007	0.887653
40059576	1.414026
46179289	0.680113
26324454	1.054276
36856212	0.896250
140782707	0.806370
41798248	0.931905
42775063	0.693487
119873574	1.460792
15517461	0.763526
44422390	0.688494
139686140	1.264607
30499848	0.638524
42037933	0.680870
30014723	1.288411
115699035	0.961352
40981425	1.946319
46361658	0.953930
41817426	0.606086
141017317	1.048146
37467293	0.683825
64195212	0.705869
65294742	1.005171
33821370	1.029955
45192316	1.500267
75609807	0.841428
30459755	0.703083
103145167	0.929466
40401346	0.803955
86592497	0.684251
81857381	0.934622
44638332	0.633401
45260962	0.797009
46436391	0.656308
44339592	1.024515
141328603	0.707079
45308633	0.766162
76123077	0.681702
32017967	1.072939
32442310	0.664214
41612674	0.645304
120835950	0.681598
140583844	1.850654
37256202	0.938194
37928457	1.122306
11398546	0.710469
64507579	1.241775
64552141	1.164973
39383727	2.008693
103187661	0.658377
35052837	0.909732
23822654	1.239625
43712189	1.470823
4199659	0.653278
39643758	1.076573
35055388	1.026013
6357029	0.670352
23487638	1.104751
14588922	0.842950
2856581	1.395154
92195299	0.756759
62294815	0.813922
105566193	0.984598
44670862	0.640874
42568189	0.643435
21382508	1.119544
30631891	1.038689
141300565	1.247932
71467407	1.477654
18884116	1.073135
16346155	0.710787
1776636	0.838778
942558	0.705449
136982459	1.203249
1059498	1.147162
15108751	0.932833
128926639	0.987290
139426993	1.168793
115378202	0.653436
126057033	1.655876
42929132	0.848771
125096489	0.989993
141038978	0.820509
17559818	0.819306
52863675	0.705358
17364801	1.661842
2903795	0.737411
39956000	1.312125
45244437	0.745856
15157206	0.831792
140154180	0.675645
140967864	0.704544
107165837	1.211448
62390931	1.171185
140955484	0.723626
55053720	0.667392
3030161	0.785279
64194459	1.303193
139315428	1.079416
44912167	0.651187
45250686	1.017605
42436350	1.087913
132038881	1.575467
18703711	0.747868
82080460	0.799464
139759241	1.238337
47884285	1.010793
32937112	0.692749
6585821	0.845905
115856065	0.957563
140316319	1.598303
138945640	0.844227
25035426	0.739273
28891201	0.892377
86127514	0.768296
15160811	0.766947
71686380	0.959594
30571366	0.878601
100310478	0.848176
44514998	1.573046
140955481	0.940938
45254626	0.823493
46699197	1.396293
91534725	0.819324
43996654	1.481448
46299343	0.630330
115538100	0.756937
38526108	0.647285
44368777	0.741064
86837531	1.256589
115538415	1.448642
17564730	1.653108
139630269	1.393621
15084163	0.734737
140899216	0.916469
36462560	1.070739
2971317	1.276989
13967124	0.602259
132356609	1.082817
13580334	0.765752
44261675	0.651139
46108638	0.769504
76123778	0.733533
14061170	0.717853
26019530	1.360483
118333804	0.688860
117181192	1.555277
93309964	0.682165
22485114	1.209102
104263180	0.795279
47878194	1.022066
15179513	1.232115
141375542	1.555195
140565635	0.784873
44317407	0.787649
34791613	0.761967
48041264	0.871022
40894437	1.117931
139573859	0.941753
41562199	1.180012
44519088	0.623446
99669376	0.770668
65247441	0.836070
33790865	1.442707
104471882	0.666051
42435803	1.102961
45290738	1.012256
140908727	0.870251
46004436	0.691975
42779973	0.921117
118010376	0.825513
41944140	1.061159
33835041	1.386457
118013689	0.848660
41653256	0.740758
115537892	0.880341
54793862	0.639249
56051048	0.963963
37441714	1.213066
14899259	0.827959
116765253	0.887283
2958398	1.240096
44408320	0.951031
37622151	0.720584
141632143	1.183163
44245016	1.046042
37630278	0.985553
38833386	1.670444
42492178	0.624849
139384705	0.667456
115854601	0.628421
131092704	1.950463
115854200	0.707265
13882943	0.654211
140474915	1.123730
134700241	0.862956
110694465	1.314625
44284559	0.837727
42546222	0.877559
45847542	0.650681
124054902	1.575873
44016492	1.326334
47873318	0.797879
39301002	1.015127
43138784	0.630227
47662181	0.966236
25769411	1.025325
90531614	0.903369
93306386	0.737699
8989435	1.287794
120838059	0.612919
39329685	1.314875
117434355	1.480002
45207988	1.566088
36287491	1.293490
115860715	0.701645
141090655	1.305114
105774848	0.897261
117183595	1.714670
43714444	0.980851
43714443	0.700935
64472601	1.174396
48901704	0.756003
57337965	0.602260
8097973	1.732092
52559101	0.652061
45121778	1.302649
116252629	1.438602
6308624	1.137035
85066309	0.734501
95659456	0.896922
33048422	1.273155
71043152	1.040711
86137610	0.670047
47056797	0.876712
63482226	0.743062
44787475	0.910320
29361152	1.216763
14615821	0.713693
38947974	0.735704
44418193	0.660958
14652080	0.729146
140172078	0.861166
44962794	0.695365
91251729	0.609649
141040532	1.002062
18819710	1.381867
44198484	0.808500
82758117	0.614667
140897234	0.854623
15231653	0.690302
40007922	0.769348
45655821	0.836377
34360016	0.743425
43949059	0.702043
140713119	1.816728
44901321	0.733632
44649755	0.713754
34647800	0.765767
48063940	0.945717
19885338	0.655391
67203362	0.728468
35344439	0.697878
103376653	1.047801
140647075	0.965580
120965999	1.470993
140830704	1.279109
140815449	0.766072
140815462	0.771730
62292793	0.878437
141631320	2.006879
4188890	0.828539
37872333	0.630258
110694538	0.676997
21265921	0.799587
141342951	0.758784
13714643	0.780657
140597472	0.875625
2924792	0.817622
116086786	1.447465
44317538	0.646048
2814958	0.887920
139345614	0.636812
2857751	1.569365
124250507	0.637402
120158063	0.844719
118203464	0.610635
17564468	0.617436
140794610	1.562597
140794650	1.461526
45207994	0.870029
140399334	0.608329
33916282	0.810893
3042809	0.847732
41844655	0.651216
90845263	0.736785
38708095	0.925346
37361080	0.629432
43734064	1.090106
44420724	0.646047
15665118	0.694443
140538593	0.632369
88701616	0.697524
45255199	1.654829
45272824	0.951964
115860827	0.612534
42415505	0.657837
26431242	1.195062
38948606	1.067002
42362627	0.696316
42463253	1.086029
116255427	1.021906
19832821	0.765454
126057175	1.050461
120966046	1.389763
98620516	1.643394
44509937	1.042269
76836010	0.899200
135728149	0.724402
140755819	0.729881
4474176	0.946146
63563926	0.721319
141352930	0.697798
54737707	0.690998
44675584	0.658919
43572377	0.669832
47946247	0.615851
28165583	0.664051
89597373	0.665235
45228615	1.237568
17574383	0.829807
6133523	0.740548
33994515	1.222801
106869641	0.630316
38303513	1.019540
47832567	1.241271
57992112	0.697038
99805625	0.606547
140701550	1.196146
45287797	1.030516
18164503	0.747751
42801770	1.197347
41037847	1.143585
38041956	1.498566
132038883	0.969370
140316323	0.936748
32417434	1.126412
44189624	0.711657
127306627	1.060276
140476382	1.407647
44253003	1.582258
140315488	0.808571
117051020	1.438011
117433196	0.735050
87619239	0.632272
15150074	0.797742
13757652	1.071308
137913829	1.141909
23347488	0.859669
45841027	0.662236
44186601	1.158121
3037378	1.998390
4534689	1.161402
66884603	0.853883
44981257	1.410471
38995917	1.251550
50706201	0.613156
115860706	1.040594
92277214	0.657914
140759027	0.948212
45533288	0.757829
3015651	0.936024
36825652	0.902650
141023589	1.107026
129075976	0.646672
141580570	1.312580
6115855	2.280576
141300570	0.997750
141040571	1.695555
120966032	0.877992
47779406	0.797776
136155119	1.137856
81988458	1.405066
140790807	1.048975
140764179	0.616531
6242019	0.755297
35384397	0.718484
141039393	1.008626
6105901	1.762387
31148066	0.665182
27821130	0.824530
2972925	0.759147
64194833	1.352908
2915709	0.683275
33234522	0.931719
33841082	0.781517
3188916	1.120749
38489815	1.011730
91774761	0.613341
44113242	1.220741
48041230	0.685878
54793863	1.510036
29456459	1.021607
119971096	0.844372
29358600	1.409966
29358603	1.032097
43139360	1.119431
141530211	1.075774
40010865	1.141069
17571966	0.791122
140326955	1.085526
16333314	0.967879
139398056	1.306782
140920668	1.261730
64194475	1.183794
44402311	1.259045
141668316	0.984756
140672396	1.782061
43723799	1.307459
140583879	0.950178
105820300	0.656094
64195705	0.888664
75237047	0.669299
30634838	0.765837
6577258	1.312443
41083233	0.601883
40057332	0.860868
71623023	0.624023
35298558	0.960781
17745400	0.725406
44564414	1.465567
134981579	0.637206
36044450	0.962229
34044606	0.978900
39145500	0.979866
73488118	0.846329
33793905	0.712871
29045518	0.753001
6115827	1.945248
2987875	0.702958
50749357	1.247573
48046967	0.827006
943577	0.611795
62176975	1.756369
109175608	0.739486
97224672	0.902939
131110201	0.614328
1773676	0.732660
14577832	1.491054
48040978	0.911099
6249991	1.480364
31072324	0.874699
39167194	0.699944
140919422	0.666012
85238332	1.353704
42419794	0.979139
39167203	0.706903
35361438	1.661548
3014128	0.928197
16732998	1.081788
16335279	1.179651
14560955	1.012753
28846259	1.217093
46498807	0.839625
140903694	1.063501
14536887	1.035335
6405362	0.804629
34067890	1.378902
139746344	0.675433
27578338	0.795419
4667863	0.807154
42749394	1.119136
120966624	0.614197
141272794	0.605797
88702241	0.860904
115856033	0.684333
140399844	0.854947
140944330	0.622428
62390771	0.717246
39304067	0.872323
866883	0.716626
39685405	0.670002
51595124	1.087937
128773024	0.688983
109175606	1.521584
47844172	0.728784
48900313	0.797907
64484049	0.698697
44068336	1.385090
43040275	1.294685
140025294	0.621904
26146494	0.988491
39301157	0.908819
15177861	1.281910
89597472	0.839737
40402054	0.744729
57992432	1.291526
42410552	0.681732
45441546	1.042753
141307075	1.023250
16120134	0.815146
44162474	0.757173
140799437	0.998854
4208596	0.683695
87300084	0.683714
42435913	1.115449
47601989	1.003211
39237972	1.494425
48013177	1.290419
45306084	0.733483
141606089	0.752433
140841687	1.347484
47832208	0.714377
18634401	0.606671
15664732	0.689277
92112276	0.605381
115854396	0.738237
18376128	0.717264
125113830	0.635876
48082756	1.157793
37822281	1.327117
63336765	0.850829
40912947	0.750292
141422142	0.818563
87300102	0.857188
43039813	0.618366
83641863	0.792543
98633439	0.831825
41812060	0.789214
32891499	0.645347
44840898	1.231775
2926680	1.190554
46183789	0.793046
89077164	0.870145
13866531	0.978636
90746004	0.957953
2964610	0.621117
140565751	1.884811
40985818	1.922014
118013500	0.664781
32152287	1.113553
116566198	1.296346
140944664	0.780793
40893428	0.735606
45291506	1.159652
216167	1.212563
44084294	0.823838
117279168	0.815208
74702944	1.136480
44589020	1.227134
34985779	0.688151
141113351	0.610595
47827109	1.043255
88699166	0.946073
121376830	0.645858
2999889	1.269504
18575383	0.771121
39664530	0.713222
32567031	1.869156
28102199	1.019627
1649525	0.710913
40330372	0.802596
139879322	0.968459
84077459	0.821885
26263443	0.828095
32704945	1.281074
56804431	1.181188
39136960	1.135633
30583832	0.753154
44252933	1.008698
44199442	0.611723
140815488	1.738835
33731376	1.011122
14187416	0.653314
39638744	1.266532
105819657	0.743293
17571127	0.912295
89077072	0.839013
137405770	0.741216
40401360	1.037457
46997216	0.771713
72535634	1.358369
43168237	0.698382
55874446	0.975959
46179282	1.469997
45607467	0.815173
64002191	0.607314
44290624	0.619301
32067365	0.650827
105826827	0.863934
41455250	1.181242
112613927	1.004025
45290792	0.809011
4180372	1.201278
36855945	0.962045
62292538	1.424590
35055386	0.799503
33901156	0.734923
76989550	0.750184
117433220	0.852699
46306192	0.692678
42898968	1.098822
140324428	0.879425
139426990	0.820427
45857596	0.692621
119875257	0.604725
47946165	0.728606
44360623	1.039845
141345206	1.313507
1051036	0.837526
28891356	1.412499
40893426	0.630146
33832429	0.841023
2965365	1.081580
63890474	1.637207
139319732	1.375982
2982301	1.493186
4184928	1.141892
62176781	0.855386
93434816	1.871523
83641763	0.787311
38709252	0.860809
18904740	0.742432
23623125	0.640747
14899272	0.962553
6288016	1.049788
139315213	1.623894
16435659	0.790509
29020460	1.051550
42399633	0.973025
42746989	0.810516
2881582	0.985765
112613854	0.606768
62177415	0.678009
123758495	1.944194
115314212	0.901576
64400166	0.752771
13993826	0.618189
42614139	0.691085
11367795	0.743511
39668377	0.610953
2985910	0.711704
8234324	0.752148
100439836	1.213423
105332797	1.280546
45120163	1.179424
37191723	1.146849
29977290	0.836699
126062826	1.319853
118009849	0.916183
127306682	0.722316
42785770	0.720150
41640988	0.984386
122560788	0.978421
53304973	1.043704
1685206	0.649870
33793671	1.515061
44965247	0.617552
140966388	0.668574
18741386	1.247915
29487928	0.691552
44169854	0.840427
93684987	0.642870
14220937	1.006015
140783870	1.242798
35361456	0.975882
4080166	1.144368
106765535	0.651284
32315139	0.769582
94946759	0.838334
30595693	1.090307
134030404	1.152504
17438342	0.730141
58162410	0.912125
42651858	0.890274
127658281	0.795393
136589151	1.096799
90889262	1.374905
125097521	0.655794
35415563	0.791926
136154470	0.766447
41676584	0.667320
26263643	0.805913
37768233	0.940519
2903796	0.766939
35492893	0.865593
6047247	0.830909
47858969	0.988128
71121782	0.624053
41103315	0.853769
47844153	1.174787
106355264	1.000931
44161537	0.711911
39906213	0.616427
140306511	1.586757
32184686	0.836099
6121637	0.706254
140475528	1.023866
11424618	1.885353
32746675	0.735083
3666479	0.830659
117434858	0.720096
66396073	0.753363
37619360	0.617467
140091890	0.778865
30638062	0.833035
71804497	0.628536
141072706	0.996703
18819573	0.862710
91086943	0.988101
141222166	0.863296
38052399	0.980119
25993305	1.003068
110324923	0.607758
17591277	0.642751
41036568	1.183554
140943295	0.812860
28756725	1.092720
115533749	1.177418
44202833	0.649979
39301083	0.654925
3000568	1.406459
42399649	0.626707
102077026	0.949380
15196440	0.737446
140172042	1.722651
28941806	0.708867
14570968	0.977980
6373260	0.723496
2969970	1.337232
141263932	0.625717
62292634	0.956713
50200938	1.525248
45473280	1.089520
38489371	1.242962
57293371	0.626066
15585707	0.982735
30583836	1.007116
64485568	0.836245
93434846	0.606301
75677756	0.768671
109403214	0.903327
91429790	0.754665
43867595	0.858850
140513122	0.623719
90524646	1.242798
140943364	0.627876
86909935	0.614911
4138423	0.840898
33913200	0.855305
86811393	0.681443
4579474	0.722430
117434800	0.778162
39233909	0.736003
47363576	1.101770
139395222	1.580321
95105406	0.687103
33998998	0.831203
32350854	0.620937
40007974	1.181855
64219636	0.670653
102285526	0.661604
34358847	1.308811
14572008	1.629540
45063848	1.059038
44932951	0.767422
47931832	0.974622
47877873	0.686599
39286673	0.619923
128777257	1.389469
140475300	0.675092
90075941	0.673557
45236116	0.665189
93309965	1.661718
39556999	0.696502
14610431	0.605247
46871873	0.719523
38210747	0.734820
74218030	0.802336
121618976	0.630965
15779703	1.147177
140315210	0.699339
47664618	0.657622
140908584	0.836256
103554492	0.739095
36608760	1.363059
87300313	0.804402
2815768	1.434938
33071970	0.728041
43833518	0.663821
42435727	1.218242
36810402	1.125540
46304221	0.866916
43961976	0.865745
36616032	0.771979
37630317	0.736806
15585669	1.756877
33868413	1.726602
26240691	0.687584
33868412	0.822893
37630281	0.872370
124054847	1.003757
15551911	1.499422
15270268	0.619338
14484471	0.894737
14576619	0.892960
5988875	0.620303
14238353	0.609093
42116540	1.473521
131092396	1.085868
117182589	0.683136
47937580	0.766895
117707766	1.193200
16197463	1.396958
63876700	0.683145
120965996	1.443349
125776864	1.240982
47838313	0.841476
85162922	0.934571
19903627	0.985168
135202488	0.616968
76042919	0.612091
35189219	1.461829
8910452	1.025670
90057184	1.069825
17209832	1.094968
65263944	0.680938
47913949	1.025915
123129636	0.777982
137995885	0.735044
33726179	0.886604
101383983	0.774246
141043122	0.762695
29134898	0.758107
41798218	0.670365
4627571	0.995741
29012845	1.101292
140178506	0.716629
6096943	1.322075
70645035	1.084219
3851358	0.621274
39187153	0.882983
45167488	0.608796
42519205	0.808511
48034018	0.663272
88699175	0.916580
26324952	0.811813
32173848	1.055007
88699164	0.685976
16511748	1.313858
3018237	0.949480
9198062	1.317559
63876420	1.628747
13916316	0.850883
39267899	0.900866
15725953	1.505737
32261645	1.171778
41783448	0.903477
141040534	0.857966
141377210	1.472366
45064318	0.645925
44407191	1.005672
45575604	1.122213
129083605	0.879120
39179735	1.034604
139567159	1.188484
2938032	1.231810
42330620	0.939087
134029165	0.678365
32387032	0.654278
4123429	0.802416
140699096	0.670104
42435605	1.073890
127670832	0.686781
46183217	0.631718
38525389	0.799182
48013264	0.617086
48013073	0.728887
14308030	0.768754
127988974	0.612519
92419122	1.514640
91774935	0.739239
64190921	0.642059
141101180	0.872067
41002804	0.699107
27824866	0.600989
32377990	0.666630
39326912	1.280888
57992119	1.787392
18693656	0.815790
90057971	0.735803
140506084	0.620033
88532328	0.839553
16142283	0.722951
18146147	0.932793
44351974	0.680555
63920035	0.959856
40330448	1.812001
117181166	0.755403
33790980	1.008980
37818218	0.844225
39905406	0.636817
28089069	1.004787
121090501	0.630923
19785083	0.621664
44239836	1.169022
140567235	0.780280
84589907	0.831605
84590166	0.676599
33791092	0.804688
16765260	0.721504
61713582	0.985609
30571266	0.978987
21382589	1.374337
140450684	0.657330
52971650	0.815731
40423901	1.239859
31772055	0.745976
117707850	1.372635
15887850	1.307501
55430281	1.734004
34705786	0.868095
96727016	0.615026
4900075	0.778941
41562428	0.627024
116253970	1.150398
39271388	0.698860
2971662	0.837215
46102586	0.728528
11389782	0.862115
62033156	0.978613
117501697	0.628895
30532906	0.913018
126057070	0.954592
54424878	0.663704
128773097	0.644159
44079271	1.363648
4900036	1.239360
132138690	0.696384
34453685	0.607771
44319458	1.042769
127661797	0.818127
62182845	0.605320
45564901	0.697079
64484205	0.889566
2990859	0.823989
30583857	0.709429
64190090	0.726016
2842493	0.750577
31411228	0.631712
35480890	0.810077
139828391	0.684854
40891361	0.806652
2857013	0.668003
25993321	0.776460
87483663	0.667467
36049763	0.655492
83211045	1.579016
40994430	0.695750
115854319	1.301793
45243578	1.119983
62090142	0.780304
16472514	1.034168
46350737	0.650077
30698849	0.899790
17438208	1.478343
140783696	1.285288
2999851	0.729601
20423885	1.134653
2830565	1.129157
2815102	0.936087
40330297	0.698927
8280297	0.656131
5706590	0.729700
45315512	0.668555
42436319	0.751684
6372460	0.811839
52094474	0.645443
33808307	0.784452
2988301	1.223215
2869280	0.964562
35313894	1.493457
38052089	0.798814
95128594	0.863698
141056560	1.766758
45192043	0.754540
102981957	0.848512
54105856	0.681246
129669936	0.719381
129669934	0.852173
47918871	0.868118
129075846	2.023753
121084563	1.935488
16286392	0.796220
141527319	0.759975
118013543	1.005352
46298732	0.939925
42292791	0.634807
2997735	0.775546
45401811	1.514753
55874335	0.612308
139432516	0.652046
35363948	1.745363
18775928	0.998943
140831353	0.622478
113483680	0.670457
41845277	0.666751
15611891	0.942998
41027227	0.645609
13719250	0.800468
45260529	0.954920
104892906	1.007643
141093763	1.361542
75508352	1.811595
45244144	0.717014
17591394	1.360670
45260307	1.014787
103375442	0.807078
15407944	0.957544
6972140	1.029207
141377249	0.616683
43440206	0.815718
16909227	0.788999
19255052	0.930423
18958611	0.996976
105566384	0.951123
133644171	1.304938
39668384	1.534378
18736213	0.810999
64216140	1.183856
115377668	1.895346
41082848	0.665259
6051066	1.101557
141002029	1.198833
140509923	1.305509
141009511	0.867557
70645790	0.911537
18144603	0.924145
2956577	1.090552
44927863	0.954756
46362160	1.081689
2859881	1.161993
3511585	0.948293
47815429	0.849170
35484566	0.651909
95848034	0.976019
115695020	0.981903
47878028	0.721012
93306390	0.611536
56047571	0.645750
141288954	0.921135
27131751	0.732003
32069375	0.885229
38883904	0.668194
18822489	0.726004
82884229	0.701951
17470077	0.711385
65264188	0.661516
91774928	1.220653
14235283	1.116758
39192510	0.644793
93684686	1.419131
44245310	0.938318
121623503	0.885955
30625555	1.262535
39262446	0.761577
140476030	1.198428
2830714	0.888029
45528905	0.880819
4140941	1.417595
15074808	0.966939
115377721	0.743196
40419876	0.641043
46284823	0.741687
30580941	1.283471
41656216	0.698146
140832302	0.615026
1909168	0.899962
34803082	0.702799
2968430	0.766045
16431226	0.966853
48841988	0.750382
44286803	0.885705
52692622	1.558615
40134595	0.781375
8163903	0.790651
42297749	1.426189
14572187	1.101025
37283807	1.166064
62295246	0.614144
117752533	0.996109
115538695	0.805647
13912989	0.613891
3061197	0.926809
5717579	1.545023
42221483	0.913175
139127576	0.774061
43130796	1.108608
2781666	1.281730
141022268	0.728799
19153302	0.736337
128308608	0.972011
57602193	0.824017
18493354	0.855455
100670131	0.743391
91775056	0.669401
2824705	0.887807
102241041	1.012432
29017815	0.681697
6131035	1.255723
6277596	0.744796
46108635	0.724527
27341200	1.047411
50253172	0.785037
115854211	0.990214
44172463	0.648803
115699400	1.677687
42486381	1.334669
54525720	1.492194
46488822	0.691538
61843754	0.782170
18690199	0.994289
45306057	0.710670
3014454	0.974702
28893334	0.614633
32018053	1.305200
115854142	0.732031
131762020	0.928905
115377906	0.668836
27339184	1.007240
94823701	0.810910
138067919	0.797269
140919436	1.162570
72051018	0.865849
23486967	0.761151
118543009	0.820822
17409218	0.677952
139964802	0.807029
115854132	0.701926
42435764	0.809752
44042762	0.981309
116765257	0.730346
16506409	0.756677
92195300	0.824780
14652547	1.136054
44636790	0.696587
34351880	0.658202
41027547	0.873111
33808912	1.653857
14669831	1.748108
139244652	0.853981
21265508	0.737290
91346251	0.998357
33790979	1.221997
30595733	0.736431
87884255	0.709994
45113718	0.657337
33806773	0.972162
16747414	0.902345
33729408	0.609724
70499012	0.871538
121072114	0.638129
83646297	0.740362
115537995	0.751186
86993725	1.337732
38763351	0.616647
141021796	0.837982
29395785	0.611949
82851960	1.105104
32149147	0.750900
44336666	0.632590

36616186	1.152726
68156205	0.725337
43350527	0.631881
19602079	0.675543
19602017	1.550083
46364198	0.747129
140315195	0.764073
41920104	1.042739
41556602	0.694322
43020973	0.878161
30667082	0.885705
24841564	1.314413
118011573	1.293773
42554991	0.600247
7784464	0.821476
31077184	0.673122
32276503	0.927795
140554980	1.007345
126062817	0.774579
15184791	0.928297
32018369	0.977265
88699160	1.082994
62035806	0.863711
45406483	0.643838
46394565	1.248959
2815823	0.844586
82977002	1.239303
29197107	0.874016
47543141	1.221369
61713583	0.910412
140835072	0.756859
101323725	0.839918
141642931	1.035755
41655744	0.747811
88699179	0.911576
32143758	0.823763
126056857	1.446061
25991673	1.401241
5750760	0.613051
62427943	0.738367
140583181	0.662203
30070312	0.774077
23356420	0.972170
43067900	0.670733
35146152	0.601146
44365604	1.177028
120966039	1.839686
139652671	0.933684
139728506	0.791841
39233926	0.610446
44161533	0.680249
4108761	0.790970
2989942	0.632288
141003957	0.847734
40994320	1.101410
39097084	0.806323
2844005	1.260730
96405889	1.341032
2835223	0.622522
138011847	1.092926
115854568	0.626482
47602052	0.835652
2825772	0.929309
8285263	0.642657
5639670	0.979551
135202276	0.858038
140815665	1.300335
23747245	0.918455
44420698	0.985981
51979710	0.915686
140815607	0.879018
29021236	0.773629
88699167	0.923779
91960407	1.940777
28989515	0.694966
47929926	0.956375
26324695	0.979306
15798980	0.919445
64514527	0.660885
90031135	1.423713
42666959	0.962502
45427364	1.178293
55718993	0.918651
121084568	0.685950
100607992	0.647080
90911240	0.880611
18739448	1.257060
56051446	0.929212
28753067	0.715742
46305601	0.614616
118332719	0.730160
15343711	0.848916
73479341	0.758846
45291333	0.969926
2840203	1.331295
15532792	0.922595
140646812	0.647591
141023470	1.022545
140701771	0.859393
1536600	0.738440
33735156	0.739180
140475421	0.745703
48983279	1.460953
140222880	1.252645
64485564	0.868711
38909156	1.286591
89077114	0.721276
40977459	0.641153
27007869	1.704110
23459989	0.615625
32260871	0.800762
65291818	0.747639
44494927	0.618389
26324815	0.973554
116252417	0.947689
121090413	0.874862
78337917	0.899658
35127694	0.729332
2829669	0.719854
53944769	0.854260
23972111	0.704458
90057166	0.684957
2868898	1.839255
102825851	1.162368
65245059	0.765361
2816881	0.628457
47822154	1.399256
100539409	1.171527
36842390	1.273578
35374479	0.882780
34587679	0.620352
139247149	0.847810
62244301	1.292921
141093731	0.740149
92100561	0.602635
46092172	0.739952
71883688	1.195324
4203472	0.671891
6097136	0.943633
41744734	0.842541
137784160	1.063319
74211647	0.679164
44245370	0.644822
18819355	1.246954
44078292	1.224424
140897729	0.849529
140926593	0.945804
56804424	1.196514
141707190	1.643954
43280136	0.738388
132638124	0.704472
64400231	0.823368
33841041	1.324517
46361006	0.729994
28678432	1.129091
35360309	0.740986
13580326	0.813846
78478076	0.638342
46146064	1.538748
2951756	0.809123
115538129	0.696738
18766303	0.624576
3839315	1.131670
140585175	1.184138
1538566	0.640585
40891239	1.126544
48032976	0.617142
93309826	1.003734
141291045	0.783726
39976201	1.027580
90431653	0.604505
135202885	0.813044
48034045	1.426496
41501655	0.840396
140013822	0.914945
37846250	0.650291
64513847	1.096590
36495918	0.824693
123126772	0.634596
15237167	0.955021
64483480	1.410213
30595701	1.096372
32705215	0.794925
40050804	0.922026
14569486	1.021676
140316270	1.096606
45160630	1.023598
45118884	0.650675
15588050	0.852580
1615473	0.616283
96080213	1.457704
35357774	0.962355
55874650	1.185818
90057187	1.399572
44381471	1.021592
37388896	0.705298
2905083	0.648712
141305475	0.934284
140815718	1.105631
141307011	1.168773
45575700	0.862184
35388711	1.593619
21491631	1.196096
4623891	0.818262
3336089	1.125042
39734345	0.702401
15779766	1.521639
37880371	1.054441
82667785	0.671915
34196412	0.899778
116251931	1.123965
28089168	0.652965
28906967	1.102764
40981520	0.824582
22725164	1.091790
119073237	0.873520
82884211	0.814391
32886796	0.670357
78337837	1.110820
45966384	1.268992
87299520	0.610908
29852803	0.733793
16760798	0.689042
70645074	0.629571
62033700	0.767975
15573856	0.738173
2915318	0.704823
45161542	1.464071
40007562	1.037162
30462365	0.893761
117276194	0.622847
64923431	0.718827
11459353	0.976653
118777114	0.661536
140512058	1.449692
48063388	0.752629
44583237	0.797092
44245313	0.899347
140529395	0.681406
86137609	0.644349
13235435	0.687541
44196834	0.632470
77500219	1.127229
4158821	0.956930
42649889	0.609878
106524838	1.114992
43019516	0.803887
18143605	1.297539
39304672	0.888742
45409687	0.648071
41786343	0.810900
45790606	1.328472
120838002	1.558416
140559446	1.060050
34909629	0.936410
34993449	0.750387
120966044	0.990708
27627927	0.621927
19502647	1.574119
140538790	0.662173
21095490	1.274613
120966051	0.600014
15764134	0.630614
116577685	0.913917
42785135	0.650155
140609796	0.931934
87484336	0.625277
45208022	1.073229
140584210	1.395141
4594760	0.787966
45315475	0.830550
33798480	1.177157
38902725	0.763120
140583414	0.760956
133631661	0.702084
72270991	0.946513
64002549	0.696069
140521581	1.446984
32262693	0.709498
45228359	1.301451
40013439	0.608856
89772034	0.863453
140780773	1.264038
48082758	1.062220
36849007	1.244271
30595680	0.991368
140198169	0.662849
14577644	0.729208
32184662	0.685606
37822548	1.069293
26750412	1.164803
21316439	0.859419
17569495	1.174783
115378005	0.882242
102077259	0.855675
61120808	0.832850
16307343	0.759554
116765100	0.615630
44337094	1.729207
23293949	1.473004
14978419	0.972051
46272834	0.864124
31576421	0.927843
28723738	1.090203
2869405	0.668840
62387959	1.473738
35344425	1.735371
36604991	0.608554
44317525	0.947119
46272635	0.805606
3006774	0.982849
64219648	1.060604
70770792	0.882443
25591752	0.715131
115854327	0.691172
141300602	0.800770
6057159	0.612415
30531875	1.047395
45244107	0.622232
15454189	1.062219
44725892	0.891307
23982832	0.821557
134023598	0.713931
30212160	1.453501
1750838	0.789656
40414953	1.247339
39162720	0.638592
61152055	1.032009
125096578	0.606356
15550567	1.403798
125776863	0.792212
21383149	0.959113
44902390	1.031020
3003248	1.288501
18404125	0.745846
16926925	0.631618
140505943	0.740887
15812507	1.021492
140188802	0.809193
42400809	1.027902
62182862	0.820230
45173977	0.828029
1754333	1.238838
35298470	1.184583
37915385	0.941969
47092996	0.787465
64511184	1.169443
38896825	0.704318
26353308	0.948143
64190108	1.167947
40788873	0.750914
44472819	0.693901
37481021	0.810392
119338402	0.865234
39140518	1.086744
36031040	0.918316
42420014	1.324795
17995686	0.933398
37798781	1.028094
51913975	0.815920
137783653	1.154804
15618808	0.945471
35152636	0.638327
46455287	0.711426
37143022	0.645444
62388943	1.048896
42116483	0.705198
3007667	1.275012
44787343	1.430950
27328328	0.854447
14042129	1.016066
44902680	0.980078
4165997	0.650019
3079386	0.817661
85749101	0.839613
34502940	0.711879
85007843	0.916498
41655115	0.909522
33815357	1.045311
38254538	0.983162
1539599	0.880773
141073433	1.405078
91774417	1.114724
5750359	1.259355
11394095	1.010278
140712500	0.768519
43646714	0.628136
14729252	0.786313
98621405	1.048616
87501157	1.297948
14031018	0.902704
43835953	1.082777
11362717	1.658810
47760261	0.617171
140195301	0.720867
85238350	2.387729
46483278	0.759315
85238799	0.633632
15407955	0.943443
39668393	0.858161
36318631	0.724895
39956327	1.151040
31408804	0.735649
141023655	0.633782
140951334	1.081896
2915716	0.990991
17438708	0.698498
89077066	0.740390
54231608	0.603332
52691598	0.922444
23492650	0.680827
37279695	0.813722
44042089	0.671820
47355819	1.317768
62176967	0.877598
99799287	0.786114
117707588	1.451860
42039012	0.708841
140793916	1.638960
79194642	0.791446
15143983	0.739737
90669015	1.378597
64495844	0.732859
4199682	0.613954
73149104	0.657844
26453813	0.895809
6124882	0.684803
140783746	0.633645
34043395	0.818314
53305040	1.212301
90162414	0.851231
51866312	0.734047
94624632	0.629765
35428708	1.537916
2987581	1.017274
115855961	0.607201
64504699	1.224541
81862832	0.701734
42462740	1.401508
44410051	0.874351
100148555	0.802084
141771312	1.217285
139759245	1.798758
78342235	1.002643
20900741	1.189191
38546767	0.774184
115856005	1.359445
138706697	0.625794
31366297	0.847017
20932092	1.709311
35369955	0.641096
44360330	1.120502
62387960	1.123000
41022376	0.849590
38891832	1.213388
140897585	0.617055
118013501	1.152198
17593606	1.121718
45235932	0.661205
90845264	0.693798
45290503	0.762619
2877284	1.396973
2876472	1.341180
2982108	0.849736
96717475	0.936508
43976078	0.894583
63875125	0.980085
30595676	0.891672
30643676	1.007284
30608718	0.602024
71039001	0.983429
18617069	1.853771
140919308	0.654058
48033984	0.697298
82193766	0.745535
63760281	0.897744
139239746	0.982379
31983702	1.134058
116252229	0.792288
31983728	1.089186
32259838	0.784022
15026040	0.826306
4534687	0.705320
85749107	0.687807
45581596	0.682452
15147438	0.955451
41611913	0.792980
29346690	0.638179
116565366	1.308715
2843921	1.187610
44581276	0.808017
42649884	0.732204
116252427	1.496127
14134027	1.259382
18421439	1.407666
84077395	1.110994
34462385	1.380407
117433227	0.877791
91570720	0.679943
140583746	0.799864
35140806	0.981274
37630259	0.980258
8237173	0.724633
55874823	0.653056
89596652	1.346583
21242613	0.886524
89596438	0.972804
139733547	1.062672
115781136	0.655725
42462088	1.433994
40973259	0.798176
140831958	0.631558
2942979	1.241840
44546660	0.899165
45306214	0.674061
3337053	1.224764
22142202	0.601426
139722341	0.839369
132138533	0.728786
140396651	0.704158
64002320	1.276938
96907158	0.671585
138705547	1.862169
42649868	0.853355
31178072	1.256133
140506265	1.406383
48063933	0.719195
45573532	0.603019
45966583	1.277304
123757223	0.829619
40934771	0.624019
53440572	0.735988
2969350	0.701644
32174032	0.661653
141087054	0.651349
48064238	1.118359
131762761	0.648408
35053904	1.156584
140193056	0.879383
16001048	0.638419
45255286	0.603598
127166647	0.660683
13967139	0.877580
43826876	0.987787
74215672	0.625902
47866327	1.225038
45230347	0.810070
29394687	0.606087
2984425	0.891648
30971984	1.340373
3219236	0.770581
44902389	1.258480
139652350	0.662831
32557606	0.815546
140815527	0.644584
46305250	0.945923
31058969	0.775870
35332720	0.900058
2990979	1.902153
140559534	1.090021
45244499	1.277452
51888930	0.790222
89596435	0.613937
84741604	1.134139
42568131	0.831904
37619000	0.740830
30674217	0.618614
44902670	1.113808
38021941	0.848538
14116500	0.898329
42774308	1.184136
39096994	1.371785
140979463	1.377739
45492859	0.877515
39136858	0.612017
6603784	0.755141
83856126	1.484896
45100805	0.904271
37880382	1.002119
10253867	0.676588
47838316	0.618898
83853127	1.034773
84077776	0.795212
120838089	1.423358
15570388	0.648867
6916554	0.890066
52490215	0.907376
4427855	0.717292
47168412	0.974671
117181756	0.857130
20125874	0.633678
2944708	0.949546
2877002	0.825599
27655845	0.793633
139126207	1.104059
32304176	0.691069
33804757	0.888030
2894823	0.880462
47905003	1.082128
16197131	0.777188
37281690	0.613233
91775030	0.739277
139496184	0.614773
15149792	1.064355
2894685	1.539553
44513244	0.633245
4199878	1.055710
2862031	1.359545
40941647	0.852341
28511059	0.920041
47544037	1.312600
72334223	0.641266
58108918	0.660410
44865197	0.812382
4203712	0.649966
123756991	0.662485
19674573	0.850109
88972510	0.977800
138067765	0.698678
14592211	1.529063
47877964	1.038598
30536864	0.993068
32809431	0.609741
10416618	0.689093
82986104	1.277288
44402874	0.842393
45261134	1.007236
56051310	0.764511
140183539	0.643197
23321619	1.272557
42541544	0.802184
37463508	0.879232
139391713	0.638996
73768011	0.950191
45087469	0.752542
30305964	0.912048
81483256	0.943360
45290463	1.452637
32290177	0.707539
139314467	0.862613
140783911	0.721651
134700248	0.641161
43965980	0.911784
11398392	1.050559
41597015	0.778343
39188297	1.183341
28896025	0.976129
45288147	0.783718
103554772	1.190429
98638854	0.941632
88866038	1.069303
36619961	1.676472
47919163	1.431649
40001408	1.060594
120966071	0.847104
2831956	0.911946
3032618	0.761531
24491029	1.221466
50253120	0.709042
104131814	1.073189
16361317	0.980759
41551664	0.693928
141842803	0.934918
82851750	1.109240
137807060	1.578665
2945121	1.669599
45336234	0.610742
6597824	0.696784
104131813	0.910433
13955919	0.797214
14874334	1.108418
15178591	1.132739
141288704	1.266023
2921895	0.616475
92195852	0.666606
42373554	1.115239
135535237	0.824391
117707613	0.949367
44902679	1.005802
56051309	1.018160
140324493	1.015650
86130531	1.006699
140583105	0.606782
11572338	1.117568
44519614	1.124498
29152678	0.656381
25830377	0.787564
42813257	1.014766
2936321	0.859893
140831956	1.415887
117707714	0.782842
30571383	1.262268
140954639	0.896144
89396341	1.221138
3850775	1.276207
6124924	0.863415
92335624	0.774745
46183795	0.698400
32801607	1.041741
30674192	1.063752
36896860	0.783071
1545413	1.045157
47950382	1.225834
3661117	0.613817
32519847	1.053195
2843914	0.642752
65249306	0.959530
38897685	1.892015
107063061	0.667055
70738345	0.690763
16472589	0.632675
47004896	0.679007
39697169	1.280782
34001087	0.981167
37133864	0.941041
39905261	1.105457
3015013	0.758249
44492931	0.699520
41054155	0.610260
2953140	0.604074
116765424	1.083778
27644357	0.678158
137659024	0.627558
2904152	0.866529
2935427	0.737407
27697678	0.791509
39246954	1.002630
47919170	0.875752
19247967	0.693856
30985281	0.944160
36734886	0.791514
90635551	0.756228
75601561	0.892451
104707505	1.176578
3123484	0.927014
140315525	0.838982
117707671	0.856832
141023616	1.314666
47723308	0.656029
40912943	0.730549
42433177	1.215712
123877264	0.603396
2919744	1.316710
15314549	0.684488
18820405	0.838020
42030646	0.926574
19904718	0.983289
42030606	1.022751
32560867	0.930443
140495305	1.559633
47877760	0.631680
30488769	0.929804
117181783	0.801191
64513637	1.073498
30944932	0.729533
15122406	0.721355
116293527	0.623503
2996336	1.651327
104470366	0.632978
21494093	1.130456
42462083	0.810530
30938583	0.964177
35339272	0.805056
93306812	0.629360
28274820	1.238191
93309967	0.735283
45272206	1.190501
34130644	0.941590
115698883	0.678864
45287872	0.691167
48046789	0.667797
135202929	0.882561
26951513	0.724533
45260283	0.891153
31262489	0.829704
43818357	1.138647
141287953	1.197481
39664466	0.882039
140765555	1.394124
118333563	1.200873
70425550	0.854983
39664495	0.882039
37879714	0.662201
55874709	0.836476
42447458	0.811385
91774956	0.832796
141038442	0.996434
28438360	1.260393
18137998	1.128814
140815532	0.608047
39268551	0.820799
91775177	0.704654
46427014	0.722917
83856127	1.157211
37320796	0.711383
135753514	1.132438
94634695	1.471871
6374311	0.943977
89077240	0.624125
65239632	0.823010
33790967	0.631237
33791172	0.679179
32281143	0.687074
91775247	0.620561
2843925	1.281619
5715342	1.363312
1938686	0.642230
2917615	0.664499
6124664	1.133601
44232092	1.293572
43975997	0.709472
28943808	1.390079
131764069	0.603045
45276375	1.115559
64190344	1.411854
105332804	0.742248
54335463	0.865818
42435568	0.778895
86989167	0.653131
86989176	0.991094
64788216	1.245312
39352362	0.729678
139392605	0.936351
140538647	1.686345
44195497	0.893711
117707805	0.653034
40658467	1.056573
137995916	2.117753
140472409	0.657570
39668155	0.714622
116255391	1.111716
704544	1.179085
38038500	1.567344
139551315	1.161518
140646874	1.177706
45228757	1.002471
2924907	0.759655
128120358	0.862249
141375903	0.833939
77616195	1.103850
32018446	0.984616
35057984	0.812161
44957376	0.773439
16383533	0.696835
45324641	0.942405
76123558	0.787173
47838376	0.913635
37880274	0.652142
43403509	1.059623
44261446	0.662808
44037391	0.624512
42294056	0.871436
42473573	0.892180
39568419	1.335643
141146805	0.759751
137776482	0.681427
73968806	1.380966
65249796	0.765916
48644948	1.070498
107179062	0.722230
46309342	0.980685
45327410	0.625716
140815715	1.539064
141584079	0.646101
13838574	0.932446
13719249	0.774074
15586192	0.956937
71620117	0.630609
98467930	0.780370
37438119	0.632553
16148010	1.203906
141009331	0.662510
63876385	0.688390
43914291	0.775160
83208784	0.972718
18708167	0.692886
39145719	0.646933
128776203	0.851734
140583951	0.691657
26686589	0.883866
105354921	1.155704
91348285	0.623800
30543283	0.638016
41798183	1.306372
125620170	0.756610
47769721	0.806619
45256849	0.655563
46183801	0.923534
28763994	1.268999
140223001	0.988241
33880173	1.389132
47092538	0.864197
140506641	1.342797
45244880	0.925814
92335535	0.986434
2909236	0.889803
2908265	1.794460
4490277	1.021382
140156936	1.343831
25769267	0.620946
44066643	0.993026
140873737	0.709162
28476978	0.643515
88699178	1.212196
28851916	1.661940
4608057	0.608081
45798892	0.809796
120965862	0.658918
140538452	0.932255
140476498	0.740139
32632714	0.826313
15516412	0.782861
127549077	0.792692
42755116	0.647417
5118574	0.702870
100277106	0.736417
122412006	1.201298
19001435	1.400655
34225428	0.891681
32488207	0.628775
140008242	0.625251
46455217	0.710784
31245455	1.336985
45401816	0.977268
6388974	0.758812
2843589	0.831663
6372973	1.490687
14141359	0.667072
135202457	1.093445
93434218	1.149146
40212327	0.808828
100670336	0.902416
3887968	0.958930
15527652	0.884728
42462475	1.020456
29021253	1.079950
115854451	1.239054
6264329	1.716533
13838544	0.757919
3593318	0.647671
141656574	0.955944
6016137	0.925639
72016360	0.718866
6264330	1.063486
140672358	1.068105
103134804	0.633963
15517393	0.715461
126553632	1.089080
18859792	0.608990
36364560	0.701154
15210585	1.011984
14642705	0.714188
120649547	1.126511
83200052	0.669316
94634691	0.920393
8936718	1.056839
14166329	1.819811
6397949	1.196531
141718165	1.382413
4155325	1.658362
41600909	0.606106
55172367	0.817226
89652597	0.710426
126553736	0.971555
29772206	0.964995
8279869	1.187663
42373458	0.621613
13838709	1.167206
47093308	0.809523
47945807	0.920838
11447987	0.908253
47511312	0.854356
30539009	0.915767
2987630	0.601998
43822064	0.791299
15663329	1.422484
2945365	0.639804
44418434	0.640527
16430073	0.795653
89597474	0.852226
42976229	0.811738
39646063	0.667982
43067907	1.439226
89596649	0.626091
30536271	1.198684
4185555	0.621295
4624912	0.608085
26952674	0.674955
45806683	0.688599
46145691	1.084314
8932572	0.745263
877144	0.624717
141605527	0.900567
50737897	0.902969
35323095	0.697933
47366436	1.006580
50330238	0.740063
50448128	1.085011
29009066	1.238956
39205070	1.342791
39328014	1.079170
25606849	1.041369
86593186	1.254392
45276341	0.978166
3861430	0.649003
14300701	0.775761
36899104	0.951959
86124366	0.896537
35123635	0.837711
40933635	0.616557
31977099	0.935710
140317612	0.742950
15186489	1.364328
134700246	1.394300
43954356	0.691219
31129738	2.082026
55874437	1.184461
72412573	0.880839
70971475	0.770776
91087876	0.787063
42774305	0.964462
39304104	0.783402
141038467	1.220423
42356135	0.654925
17450210	1.114241
47737147	0.655710
42435650	0.608925
4627035	0.615421
39404084	0.752683
87857806	0.717006
5694299	0.824529
128620432	0.655460
139872338	1.111188
42649866	0.936978
46786853	0.623538
45207990	1.931174
68719808	1.036051
15752668	1.077195
130960203	0.689119
39233901	0.868753
29020491	0.976379
95942260	0.836468
33868410	1.008730
56804426	0.611168
83985465	0.892481
90911261	0.987409
38860671	1.120853
114301753	0.727609
35384399	0.803787
117181276	0.686274
48040789	1.245785
45094838	0.721729
35924346	0.616602
86993823	0.809252
87950987	1.017329
141001900	0.805944
47826828	1.509002
16510766	0.929384
140919336	0.755886
26146556	0.866483
40485447	0.985440
28987892	0.789541
79318606	0.606051
90174995	0.832671
140773096	0.727336
44239656	0.896688
65245067	0.679279
38710448	1.014728
6655891	0.845024
6608108	0.835246
18253892	0.662035
45278593	0.676062
5724715	0.747108
47848927	0.668917
43822032	0.620831
72996545	0.900261
48041238	0.913005
46436327	0.791756
140156799	0.654732
87500735	0.947112
140538578	0.681341
1886586	0.847515
14497515	0.661478
38978669	0.834079
115391004	1.002102
131214867	0.736654
131384519	0.601635
39611649	0.622894
15250201	0.848862
141023617	1.410148
16022046	0.644933
42447467	0.678404
47873298	0.821253
2909183	0.865116
15617896	0.772452
33836417	0.613007
2830883	0.926353
61100675	1.399318
32209846	0.623375
41056301	0.838589
140873718	0.983407
45351459	0.658870
94503193	0.642957
46499683	0.903853
47719738	0.766097
44272405	1.162477
32037525	0.867557
18122107	0.681555
14307603	0.881893
46098385	1.145747
43238425	0.896762
88779338	0.942036
45627294	0.657465
937028	1.017292
98639017	0.896122
137645004	0.881954
48063839	1.509777
116765647	0.747665
140475706	0.791624
15355627	1.123674
46447651	0.834064
54422978	0.724854
73559186	0.890281
15803436	0.785690
139104999	1.144322
137405765	1.709911
40999490	1.112579
54192026	1.254107
120835463	0.731335
3061245	0.708781
33793676	0.656988
141023582	0.871157
41621636	1.386703
14689603	0.885279
140699415	0.620664
45236402	0.740345
35359369	0.997813
39105457	0.773350
73972200	0.997378
16411594	0.602008
6001514	0.799961
61453340	1.466374
100265317	0.733266
84741695	1.158955
37446178	0.635853
17683682	0.801376
39730766	0.769601
16252509	0.805166
23487005	0.733334
2958292	1.237730
64190111	1.080794
141324044	1.278861
26003719	0.693505
45165123	0.956880
41858967	0.691749
17944347	1.001137
46179137	1.554339
30704263	0.850360
90057882	1.192464
96600685	0.887666
115854173	0.859788
140172232	0.955992
40977408	1.018311
43855278	0.731171
140183039	0.819442
46361001	1.043892
26240681	1.031402
37921795	0.700751
140717464	0.677426
32432984	1.025140
64495938	0.609099
47602047	0.650497
32673871	0.782254
71883754	0.791669
25781950	1.050120
30595814	0.602501
30814280	0.708854
40330424	0.726366
140495382	0.720024
140559464	1.013532
105821173	0.955113
18164520	0.738015
27632259	1.025457
35359689	0.815207
44492294	1.062353
64103013	0.675505
28855584	0.742571
30608666	1.104028
18922127	1.148996
18700157	1.095464
65250556	0.730067
118013626	0.824122
16513911	0.624791
102079368	1.138899
44146325	0.982179
6027609	0.880737
42672269	0.601900
39691410	1.156544
6250893	0.771836
17201235	0.938869
45525685	1.126299
40735346	1.152965
48842080	1.169954
44280606	1.756022
102139743	0.819303
42399621	0.909463
33868429	0.738142
37630269	1.132916
23806415	0.608318
72232264	1.037007
30816878	0.899065
90070171	1.242575
9197901	0.731333
2996335	1.210071
135535268	1.838399
21550760	0.832481
26767415	0.605097
19991378	1.324890
116252404	0.654602
41092737	0.707497
40997764	1.971440
37799661	0.718804
5722829	0.721827
90162408	0.924089
62387958	1.058960
117707437	1.012101
117501754	1.505066
32166465	0.842924
44807835	0.906311
140079281	0.804546
38945972	0.804960
48041217	1.127768
118010850	0.912135
54191866	0.742030
16966668	0.962327
31113734	0.614246
117707885	1.626186
42287959	0.930252
82851807	0.960800
42165490	1.016431
43596955	1.055586
43030572	0.926325
15162349	1.045776
115378004	0.649695
14636689	1.342466
140749218	0.684494
20659056	0.608118
71043133	0.603522
39321487	0.617480
117497943	0.847930
34228774	0.709575
39174162	0.615376
70777138	0.773955
120604764	1.237975
140954638	0.775504
123634544	0.741854
35428674	0.742110
18681190	0.780487
115854049	1.462074
4559272	1.074301
138943124	1.573665
14022292	0.664604
136985666	0.686830
43961923	0.642620
15475661	0.703608
112613899	1.045315
24777894	0.719114
45030541	1.085115
36832829	0.794077
40030384	1.390722
43812562	0.783025
127446596	0.768389
30923715	1.250766
140701952	0.688396
33821472	0.780622
54232247	1.180165
84077900	0.885687
141421950	0.883588
42775918	0.826081
14588803	1.129909
121090306	0.688137
18607452	1.105845
87702342	0.648662
42442973	0.967420
47859152	1.110543
43646572	1.278800
71236651	1.068819
43814347	0.772722
141007070	0.693288
35352930	0.610145
44290878	1.174697
139447880	0.715327
141307013	1.254712
42356045	0.620095
44135785	0.881513
44725898	0.665891
92843719	0.716188
23486981	0.609937
42546409	0.634832
139965064	0.670858
77307001	1.048844
47662121	0.973824
2821718	0.713386
67211599	0.655698
140026978	0.805396
47542666	0.746213
86592319	0.966622
945154	1.140027
2984606	0.927697
129076351	0.870349
44364325	1.369434
4140634	0.984548
44032830	0.941803
34182107	1.450382
37630271	0.628654
1780261	0.880878
2990456	0.774606
140146907	0.777716
41096630	0.799644
39939073	0.688898
63920036	0.607707
140841770	0.737698
91794027	0.705888
30639069	0.721643
39304784	0.814314
62427553	0.806054
30670853	0.667987
140765667	0.626464
87691018	0.811909
40065933	1.198670
42493083	0.678821
89077512	1.741530
63890954	0.831231
41817929	0.947259
44581086	0.685167
45165624	0.766732
140709935	0.876621
13902049	1.463690
44261648	0.741893
2928504	1.253042
44252506	1.609564
17470073	1.089014
34164000	0.728928
79832108	0.812964
140274404	0.823634
87951001	1.022285
121823701	0.624426
51367364	0.875451
44402120	1.338401
76413561	0.967583
2859475	0.745785
3734572	0.861945
3011191	0.714378
1788034	0.867652
140783735	0.971827
2913249	0.690542
78337842	1.172815
118008957	0.930464
2940769	1.216140
140315722	0.776679
33791089	0.760679
120835572	0.922158
141518210	1.178074
30185628	0.600668
47905247	0.756944
45077486	0.688272
96603860	0.614834
131092100	1.057022
45704961	0.615233
95130471	0.803900
116765906	0.744388
32027000	0.912040
6036347	1.126411
35055427	1.014117
86479747	0.888525
139566750	0.662303
34382147	0.833647
3020406	0.659922
44583200	0.774733
45154582	0.890446
18785508	0.840498
45689417	0.671086
4605854	0.618555
40893538	1.122022
41657135	0.846635
140583904	1.114711
140583499	1.020990
65214476	0.740598
45245228	0.939713
7882016	0.680931
140495503	1.319633
2993219	0.705401
141071455	0.694073
28861158	0.742097
90524644	0.860052
30595657	0.735646
31244904	1.209912
35384430	0.801777
32014145	0.682854
46276919	2.109649
23594626	0.652605
119397870	0.771690
66964901	0.946271
141375640	0.658329
48901759	0.754963
45034321	1.172895
140705844	0.654706
128546901	1.036260
47839267	0.976190
3106820	0.704069
46427154	0.737603
50392390	0.935148
42872371	0.732881
35127817	0.860621
32202039	0.935768
3045234	0.892968
29660658	0.632806
47511953	0.938884
39027777	0.836436
44261661	0.688734
48066793	0.998410
40578000	0.764286
32705356	1.209539
26009010	1.503522
44128207	0.866417
47832458	0.774085
42670588	0.675817
117433513	1.138418
45281298	0.647719
39182931	1.377602
30679664	0.863293
17824053	0.982098
15781156	0.793855
88866107	1.438390
44246450	0.766235
3475146	1.003375
39301782	0.853575
17913994	1.557052
140506982	0.822589
2814922	0.746974
16428396	1.158771
47773705	0.921365
93434559	1.647206
32632716	0.880369
8130929	0.932907
39233874	1.278658
37799335	0.796138
47904998	0.888373
120922215	1.336304
140538579	1.160195
31621560	0.606960
37821752	0.924906
2967408	1.384805
45030554	0.876465
40049171	0.612757
6395170	0.826243
64400242	0.965839
48013170	1.144599
141001997	1.131885
21290090	0.775438
39073321	0.728328
70770783	0.603558
45630724	0.792854
25606872	1.138341
41093230	1.883573
29015573	0.718563
44718333	0.619183
36056141	0.676555
1754201	1.051839
116765881	0.620690
39178310	1.416108
43973477	0.757508
106355379	1.283807
2915307	1.274665
2817103	1.118149
137420460	0.723912
14561289	0.660681
40680868	0.783840
37799344	0.707311
42435755	0.738917
4649827	0.875125
96251328	1.009313
42322726	0.759099
2895177	1.251427
39183674	0.618167
37356646	1.186979
42330634	1.087597
50201034	0.694539
46498813	0.882339
41552262	0.676608
76411257	0.640180
40778552	0.877631
140833183	0.776475
11296822	0.811576
45473669	0.790709
139447864	0.851993
42452874	0.819395
141307014	0.614538
141056570	0.645217
39089155	0.666017
43618745	0.627999
63760481	1.037498
47918901	0.602046
140956267	0.854912
16229091	0.614054
45039111	0.961388
135229583	0.991803
2997405	0.737686
140538731	0.949754
119341161	0.617051
90070167	0.620880
28085281	1.102900
132859323	0.686847
107178817	0.844946
41049137	0.836839
15957633	0.948966
18741384	0.627788
115538572	0.604664
26171569	0.899296
16309899	0.942215
18861655	0.671270
141366569	0.726571
54555393	0.619730
140815711	1.057822
3791990	0.953804
43939301	0.682213
43939302	0.690516
2995093	0.633413
20946385	0.760272
41605886	0.954306
11341424	0.806734
65264038	0.789880
21267692	0.656153
47953886	0.905893
3068631	0.968877
3082659	0.788707
141331329	0.850302
140782130	0.696498
120835662	0.710605
45510855	0.640031
35127574	0.639519
45520411	0.738323
139345613	0.625533
33810613	1.105251
3027561	0.795966
29881683	0.708274
40377355	1.316700
31207436	0.728360
39097077	0.976082
35493176	0.972323
140584246	0.858289
141584081	0.740211
115538821	1.165736
140749484	0.864263
85746229	0.785202
121619169	0.791863
45651036	0.663810
34791601	1.170314
140875923	0.819946
86128061	0.730092
115377702	1.432360
32550434	1.237499
79318206	0.609592
54112749	1.010922
45622641	0.729547
42217106	0.620744
38592688	0.682655
31159257	0.817251
86451151	0.701523
32173697	0.644743
64510416	0.708301
68144796	1.005706
140609195	0.721037
38042087	1.193688
134314402	0.937889
87956899	0.775612
2798589	0.921450
7785744	0.743223
2867886	1.585451
17607340	0.682487
35152627	0.613176
22556927	0.830103
71543372	0.715436
116042122	0.771118
120835183	0.865387
140840200	0.952040
40888921	0.911503
115533721	1.087029
123878264	0.613894
124967797	1.472746
33915684	0.867221
32495641	0.987112
27652415	0.813157
3011999	1.113130
45161683	0.796470
74178025	1.041164
20936122	0.979781
13875265	0.633691
45313072	0.656636
42460461	1.030199
137420372	0.726239
45351927	0.961728
64495899	0.702099
85237738	0.944556
45235906	1.374107
94958950	0.969846
116565788	1.368122
120158741	0.741925
137409469	0.667649
63633452	0.692386
32262857	0.767934
13922995	0.655402
141056699	0.698662
66255145	0.616458
41356435	0.769593
46678959	1.630184
3026014	0.637022
139246784	0.769136
36734708	0.608635
45208026	1.272511
6641533	0.873208
33998926	0.749322
14736318	0.645098
17607218	0.829287
126553897	0.632349
43867002	0.974246
38188359	0.793550
11394531	1.107412
29179052	0.919229
141147778	0.901083
34373347	0.741990
33880251	1.253158
127301267	0.697881
48780048	0.961596
87884155	0.710853
41052555	0.912496
44245234	1.527884
62292527	1.011681
72608248	0.680740
39577113	1.025163
74105229	0.831813
141106201	0.816653
98626100	0.864285
26396971	0.999800
33920303	0.621197
4686517	0.663971
13904688	0.609999
56051116	0.607678
42785188	1.447686
2912111	0.606487
98642597	0.828206
38546352	0.862952
9173164	0.940950
9196121	0.961737
87500709	0.600973
9209035	0.602029
61977105	0.780791
40931278	1.364193
62033212	0.797006
39535059	1.068146
42462277	1.067242
43998215	0.788326
29021088	0.857629
29189353	1.414455
97485850	0.775263
34986305	1.198128
44000721	0.816771
134023596	0.800246
133019425	0.696621
31074758	0.783741
136062112	0.757682
18693957	0.663665
71045721	0.725989
48054876	0.632813
36735315	0.753118
20937368	1.069277
37181539	1.237726
2842688	0.676517
44760371	1.484570
38898115	1.055750
46100930	0.708235
17534455	1.592514
39183774	0.749347
61200537	0.753351
5831567	0.794593
13960168	0.951014
2907178	0.605883
140944353	1.297569
39918744	0.988713
62244308	1.143593
15054609	0.840726
36011913	0.989660
45607749	0.985679
17394894	0.673037
64788264	0.738493
21382623	1.359653
3059851	0.740525
64514441	0.695066
39695416	0.740076
140880151	1.419767
48041289	0.876826
42413926	0.858977
64002296	0.604108
115541380	1.169637
40932644	0.747960
30696907	0.940648
24752890	0.741349
140583485	1.485108
47929280	0.691156
78482624	0.973712
18730150	0.704536
119971090	0.818769
118013619	0.996180
23348407	0.888443
30809810	0.785207
99904024	0.890527
26353237	0.913395
18683290	1.197109
32166502	1.394590
48983964	1.105144
36854330	1.019105
43067204	0.683470
31077177	0.697838
47918865	0.618572
48983781	0.759673
42331788	0.674215
25610720	0.916321

39229931	0.654102
2839462	0.611081
133019363	0.854681
3089558	1.325516
15044887	1.276562
45526605	0.747958
41651076	0.624421
90057734	0.865997
16134803	0.758418
40414930	0.619333
16934895	0.611612
43426098	0.709837
44584342	1.163113
64219529	0.603091
16408397	0.718523
45244222	0.983752
48900081	0.892749
32184665	1.061493
141009499	0.708795
14149796	0.995253
39947282	0.601753
6298090	0.634653
115860799	0.618842
39327621	0.787724
52559092	0.993473
91775287	1.063799
46327618	0.827720
45332884	0.784210
90525107	0.864434
117497978	0.668559
89597467	0.808923
47093309	0.738370
2928466	1.348606
42859900	1.222919
45345350	0.847883
16351937	0.906626
45473417	0.832054
30618169	0.728347
47779423	0.690199
27182695	1.075036
4175690	0.730785
40994796	0.977777
140712451	1.246644
9189233	0.664205
77901397	0.773427
17574428	0.665670
50257416	0.732398
15588710	0.664496
124028568	1.023795
14063223	0.699724
3342002	0.797728
63890475	1.028914
139125083	1.452042
21103573	0.967037
44172427	0.827385
120838003	0.601043
15147602	0.977620
2831403	1.723190
37846200	0.635648
2996338	1.017765
51088304	0.614644
91251545	0.723613
3078520	0.878164
32197273	0.665992
34176090	1.341286
42321901	0.639490
14736466	0.788037
41654022	0.633241
45300182	0.710174
90162348	0.634014
40320704	1.201003
46248278	0.641136
140562431	0.687004
37478044	0.739384
39214205	1.305994
4542339	0.951947
38764262	0.743527
42436184	1.393595
40146687	0.601034
5738160	0.656333
2942892	1.310998
17564542	0.601953
45236408	0.950915
45173906	1.018774
92423784	0.998108
3003958	0.808348
37619440	0.768081
17834591	0.672390
30569901	0.956094
34010846	0.738204
21273126	0.841151
107083393	0.610684
27538303	0.746807
22839224	0.644296
35359636	0.676034
42419312	1.142497
42419358	0.786697
35067345	0.672153
29021415	0.832572
135096978	0.880150
70645862	0.711425
42212642	1.521333
45315510	1.088103
29691662	0.643653
29855526	0.629061
34974374	0.762528
15453630	0.630040
28891250	1.105907
25767372	0.729897
6248938	0.758694
72996575	0.860719
19881509	1.095899
63563744	0.926170
45845594	1.081566
15557897	0.940261
41108161	0.746060
131647607	0.623556
1938546	0.940523
37929469	0.765989
91774255	0.795843
140943233	1.118058
113482715	0.681692
15249012	0.611148
141112859	0.936894
14018488	1.065164
45781655	0.856097
15203102	1.210212
47745382	0.915447
90057896	0.718817
115854357	1.089601
44191807	0.852661
41843107	0.660701
43131243	0.918533
33867325	0.997347
39164945	1.001681
37929081	0.880231
37199260	0.740070
141708498	0.846123
94958794	0.892831
140495194	1.400269
117434824	1.008322
90057534	0.655744
46066117	0.692057
2932067	1.429573
3017643	1.507517
139828541	0.747185
140815606	0.629199
2953743	1.133081
20419406	1.255020
47877897	0.677337
21088423	0.811029
118333798	0.878126
65237156	1.477277
141064343	0.656916
55862927	0.898033
123129460	0.684252
42710142	0.659021
48041171	0.606383
2838453	0.695837
118086982	0.620560
60630437	0.649757
122651629	0.611699
18611525	1.005659
47602109	0.713679
94763266	0.774026
16830696	0.947143
16431233	1.247194
2894917	1.363901
4643227	0.601399
70455348	0.981258
44469191	0.749856
140712770	1.131852
15706222	0.704367
25887835	0.715007
127449369	0.685618
115860735	1.041081
42435721	0.919479
115698979	0.625494
48049881	0.835169
50200927	1.086608
140738514	1.002137
89920808	0.696697
45331992	1.055369
16435603	0.761158
61313018	0.754271
39744041	0.765015
141016408	0.694740
47664666	0.831216
135126127	0.801086
11396398	0.683089
62388705	0.857371
17569672	0.809329
90531852	1.350441
43150704	0.776362
43309479	0.916908
37880288	0.916272
62033290	0.776358
33814779	0.908921
64546871	0.658194
33994318	0.641717
115854642	0.636783
43814372	1.007629
45299012	0.943632
44169350	0.735980
82661206	0.737824
1733482	0.620493
15247182	0.792487
37139142	0.613062
42530465	0.815451
2870877	0.752483
11533080	0.639362
42877071	1.427717
39368612	0.605058
90077984	0.633244
4423259	0.684816
11400744	0.709045
21902464	0.896579
45119236	0.714432
6130754	0.916306
63309897	0.780091
40330298	0.831690
140509865	0.649209
72509411	0.751298
45243998	2.155490
103708451	0.724845
83642205	1.411327
45161498	0.762192
37822013	0.627443
20940808	0.711066
45189711	1.324596
62244307	0.971147
45165510	0.727649
11469179	0.871661
85322463	0.982620
45106418	0.619819
140495687	0.735410
13849484	1.000906
140646856	1.229669
45254228	0.779415
69435640	0.921348
77616199	1.359053
42325075	0.728763
36319819	1.213285
41739800	1.132587
18720817	0.896121
89596822	0.618975
3043913	0.630007
45276330	0.711410
140815479	1.202584
118008907	0.789642
35384409	0.772757
21393340	0.847491
38384053	0.618905
29839066	0.859894
6918273	0.927609
40740560	0.983021
134981586	0.789558
37384201	0.642916
115084703	0.875343
38654936	0.989020
44146275	1.096639
30015226	0.714643
105821174	0.677825
40986504	1.135877
3000084	0.816654
2932311	1.075299
6597089	1.167342
36274015	1.222268
47870501	0.665504
32377986	0.884399
64194440	0.696567
19601993	0.782989
141311531	0.730650
73890451	0.756403
45646869	0.640275
62389083	0.694428
127299188	0.602341
87867797	0.860372
35313939	0.825415
28102184	1.515352
73890496	0.892200
140755060	0.601073
42436580	0.683972
140927111	0.664751
71789692	1.028663
31124802	0.883327
40095171	0.949771
37512042	0.660378
13885426	0.781691
1753431	0.999402
35370693	0.899762
19153319	0.619711
46176908	0.966617
50534974	0.680742
44185473	0.843922
61154117	0.849482
2842569	0.672086
32724499	0.637606
126056918	0.982582
90746856	0.615364
29365962	0.665025
100535701	1.604077
2973522	0.752494
3035641	0.847953
5649063	0.623207
28929564	0.612177
46384427	0.959579
91057413	0.789058
124052632	0.662439
55485655	0.858599
14644378	0.648970
140783734	0.634542
30914865	0.935020
140815609	0.963041
31409823	0.828156
3076609	0.930288
21471682	0.668203
41130504	0.619265
64552175	0.685963
139846465	0.678556
113549551	1.175183
47056575	0.701477
140908432	1.916051
44658338	0.676891
35138792	1.242998
34255708	0.770627
44592560	1.064524
126566015	0.609400
90071353	0.868674
45567603	0.669063
42465774	1.051242
45092454	0.902311
27641349	0.637696
28846659	0.989713
29905235	0.609559
43994020	0.961843
41328875	0.629802
42893541	1.708889
87501263	1.314079
130548682	0.814109
47938113	0.685726
34924932	1.385047
39237987	0.656865
47877704	1.066517
25908715	0.813613
29801784	0.925180
45120386	1.031671
64514384	0.639540
45254627	0.784413
60947562	0.805829
36648889	0.740365
30976634	0.703235
91775027	0.620834
114301683	0.920002
43594509	0.749035
83326963	0.650077
84077824	0.667426
46309363	0.730969
91775398	1.120551
116577647	0.647655
46432116	0.732941
35147744	0.975604
85749104	0.907986
92032286	0.655832
43675976	0.627222
14435333	0.758283
15122382	0.756391
48063942	0.732914
139105561	1.121377
22647081	0.629483
22234357	0.622751
36554544	0.688651
115312492	0.627528
46277037	0.848192
3012352	0.779041
84077756	0.631294
137271596	0.697384
48841885	0.795642
14937909	0.765602
2969952	0.744138
14260426	0.880921
15208250	1.039276
2856644	0.824341
139230684	0.825334
16153901	1.280144
42187956	0.812798
65274142	0.636159
21266115	1.053212
140077283	1.158338
140767299	0.606230
5995988	0.838290
34015932	0.658391
16334634	0.625019
32894796	1.212444
39682860	1.079417
40099676	0.693663
67650213	1.194263
44252948	0.636989
33881474	0.866566
9197903	0.609229
38042136	1.255684
141882289	0.653447
26953075	0.701318
1752989	0.815926
30569836	0.752196
5802133	1.346945
91774969	0.829483
17938444	0.899716
44253418	1.129970
45534205	0.728124
15150367	1.017200
23065507	1.158745
140938269	0.705224
25226187	0.733493
45245231	1.097048
77616194	1.156855
15151631	0.840523
28895664	0.725689
39907081	0.606824
35058431	0.733518
26140961	0.656684
141375517	0.610750
43823880	0.635686
38900748	0.607675
35124481	0.696524
42050729	0.652283
18688615	1.002932
47929343	0.968000
43371078	0.902031
32107311	0.956042
46493086	0.869776
13897693	0.905817
54831597	0.674302
1786877	0.876396
45271198	0.632304
106534966	0.673714
33918878	1.337543
107112527	0.893164
26019505	0.630347
46799337	0.800007
45672036	0.650801
2951525	0.606178
83641905	0.671594
61755491	0.732474
35071801	0.860288
16361295	0.804467
39991177	0.683556
140340161	0.992996
50444114	0.732347
52002104	0.783049
56719039	0.773724
44401533	1.018445
42238123	0.615187
140729231	1.023147
41138103	0.652377
33979486	0.872803
3318189	0.635760
118009810	0.609352
2069781	0.786686
26003202	0.708766
70464552	0.788979
35384433	1.405473
64499338	1.086498
6244844	0.634834
3888617	1.664849
130972164	1.273343
45345383	0.739822
39638481	0.621232
140936264	0.895675
15523693	1.147141
29022995	0.857993
64003173	0.671252
133374048	0.854848
42678689	0.778398
38873333	0.600352
31090552	0.837765
45165210	0.629457
42486521	0.723570
63569792	0.766440
46623115	1.617977
47889465	0.945287
140628363	0.756835
11555637	1.435284
26141432	0.965870
34615844	0.890018
64483987	1.339205
15300885	1.020918
44461469	0.769166
91535242	0.806180
20747304	0.801242
14268536	1.157066
81623115	1.139539
45282180	0.939484
99799280	0.606781
42729477	0.837080
23347674	0.832516
41223204	0.941233
103552986	0.835477
18972426	0.916978
43707671	0.788847
3797748	0.860141
39589887	0.720849
6593101	0.616396
29005551	0.672818
38264433	1.254242
62388216	1.156933
66303671	1.559917
16431161	1.590732
34704768	0.620219
45272040	0.924705
77635760	0.772601
79898736	0.756331
41798275	0.900941
44280612	0.613996
89828863	1.131527
140594905	0.683868
39368307	0.906776
61750243	0.637825
132038873	0.612323
38562976	0.743795
42436217	0.691694
48066866	0.655679
46356020	0.891959
38860567	0.670394
37126057	0.624721
99799276	0.802307
17976415	0.799293
138706705	0.959643
39663655	0.615545
44965221	0.938716
45332149	0.845354
116294460	0.984115
3151561	0.721298
17845277	0.651087
140705830	0.848234
5738099	0.933931
117277171	0.945237
5738098	0.883932
18533377	0.903161
45965434	0.756097
42435600	1.256163
53302433	0.880015
41343508	1.041717
62427752	0.784890
34180055	1.635599
41129489	0.674236
26353296	0.826350
21481671	1.434295
2926823	0.676408
74650466	1.178834
1539197	0.690239
26655202	0.704524
18739461	1.093846
64519512	0.895908
1752758	0.697364
75668132	0.701289
3078760	0.604126
70496793	0.630680
84077615	0.669897
41772468	1.033470
39991860	0.620901
53669628	0.819958
130977400	0.746969
44418832	0.648466
36208863	0.771744
31633911	0.870377
116577655	1.282324
43944027	0.812060
44588828	0.926318
141546074	0.895723
33999013	0.794490
2901719	0.894908
140951368	0.757834
56877037	0.636519
140783710	1.330943
83641746	1.009010
33791126	1.091457
93434522	0.764708
6250645	0.880598
41738598	0.798520
48976644	0.680392
33137185	0.974833
47993831	0.639072
42515833	0.969576
3000752	1.087575
33718831	0.741110
1750967	0.612784
120602065	0.984372
90057368	0.796272
140712807	0.805901
30875750	0.663700
39329222	0.839767
54192123	0.616318
48034036	0.602615
15174738	0.700708
99799284	1.067763
32464106	0.622868
55053721	0.751858
91346277	1.254884
45272953	0.928032
18688573	0.698622
37439474	1.013505
41767316	0.673636
37004942	0.779762
44844015	1.300251
141112745	0.608692
62249109	0.614587
40417157	0.604320
36229456	1.370488
45603054	0.606427
2835384	0.916798
97320244	1.381879
44494895	1.044320
42872107	0.816537
14873537	0.643048
50195587	1.342650
41755337	1.185052
34811297	0.662819
92100632	0.883352
44899109	0.687820
13901505	0.653823
42365563	0.615698
141299862	1.341189
18665815	1.042370
45259590	0.714830
32473745	0.775316
89596866	0.810979
48842070	1.209680
19881508	0.899355
55874434	1.042950
44180771	0.743169
86993698	0.698616
46078278	0.766378
44186605	0.898843
3726678	0.679829
13868769	0.777422
1539173	0.704303
29018480	0.608759
42775139	1.197434
81049057	0.817835
16435664	1.166616
31045091	0.931602
3028729	1.163646
41082804	0.790884
33923390	0.685407
31148104	0.632581
13877710	0.930535
30411259	0.762506
11429175	0.871930
19255647	0.624675
42649911	0.771137
2856972	0.624356
32127274	0.779716
40904739	0.854749
140773101	0.649137
32107407	0.731142
42428063	0.699839
44495749	0.738380
6405097	0.714726
5125718	0.685822
41056658	0.703330
64194464	0.876401
6152730	0.607216
6152740	0.938557
21459707	0.812867
34671689	0.934506
40774649	0.639478
134691998	1.926631
14030391	1.478837
45512648	0.942959
8234319	0.751732
22862826	0.818323
140645231	0.644551
45261195	0.643900
1787066	0.646269
119073467	1.061845
883852	1.063545
44068344	0.610513
21487342	0.723728
44592137	1.065756
3044349	0.853877
101812707	1.019200
40370919	0.890859
45402550	0.775484
107179090	0.654638
140475094	0.632031
8137916	0.922505
5723056	0.621363
82010318	1.038905
22741303	0.706271
141068956	0.744717
139740213	1.298531
37907133	0.627510
140167654	0.645550
140507095	0.734481
103542896	0.901101
15574319	0.686533
28838663	0.663731
38832004	1.439339
45580206	0.838474
141273346	0.823933
103559639	1.081820
70711904	1.477814
56804578	0.934405
42774306	0.998632
42613917	0.789362
42555784	0.738744
15482495	0.625453
44261642	0.796509
50444386	0.687894
139844087	0.672155
50754824	0.607607
23000486	0.846720
15989905	0.781829
88864802	1.336487
6399467	0.953634
33867316	0.676480
37916730	0.687216
823809	0.635953
42435642	0.722393
127443150	0.935803
2800083	0.761722
106535026	0.951065
41744730	0.789888
15524778	0.632478
33799982	0.608326
90686935	0.745778
45557824	1.525231
31162281	0.631201
88779396	0.608052
4408670	0.681495
141001952	0.773001
43761727	0.627029
48013016	0.682872
3603068	0.881320
44349011	0.872789
35568439	0.616637
47015689	1.223329
44422721	0.610684
16143042	0.675715
33994313	0.938626
62089480	0.655808
125620991	1.093681
127769526	0.827827
27071407	0.954837
36812625	0.758225
43275415	1.070964
39467449	0.670414
32160056	1.275969
30501875	0.653241
15112588	0.646113
30459254	0.679909
45928381	0.633429
26885016	0.800554
119339768	0.807648
32187992	0.874920
45427059	0.615510
35315468	0.625471
20780457	0.880414
44863322	0.814629
1928107	0.618407
33733884	0.652253
44870847	0.801465
43838801	0.603093
88678784	0.951782
140506468	1.175311
15520212	0.898978
15570223	0.957342
122560690	0.730864
32632697	0.863669
45379586	0.627584
91057419	0.663784
32059634	0.628084
1053868	0.942802
54826044	0.779493
64524541	0.646164
85237460	0.931003
75505707	0.857099
5720902	0.709526
112374321	0.952036
30539159	1.012099
3032871	0.762276
30595745	0.661083
2996048	1.305512
141527828	0.660300
45407511	0.674838
140093188	0.829565
140509869	1.088957
14140109	1.391352
41012493	1.417968
36042509	0.755004
67486745	0.801020
99803497	0.815039
140720159	0.608279
140749359	0.741898
43511428	0.616926
37490223	0.969841
29191451	0.903058
64477428	0.881277
41210566	0.629185
13434483	0.651449
15524144	0.735383
15210620	0.651566
33799921	0.683223
132514186	0.806803
15166102	0.814078
83060524	1.349743
42463486	0.994830
81052457	0.619686
2966697	0.696611
20760660	0.724055
34355738	0.949852
34682857	0.687312
3070296	0.777731
82138818	0.619462
32660860	1.201133
46427150	0.743386
26456630	0.725604
91567402	0.807135
2909166	0.736198
32517667	0.761738
44956639	0.622576
36736197	0.803850
36264290	0.772303
4498696	1.372257
31510695	1.029477
11416561	1.368149
3958890	0.627325
45228106	0.710340
88702402	0.993740
2904030	1.252049
101813410	0.820778
139828402	1.246938
48091441	1.220806
90656682	0.721844
141093750	0.609884
62179542	0.682111
42805785	0.663402
140979712	1.129920
46145600	0.746246
14266081	0.843703
68896217	1.061365
47938163	0.681506
141553977	0.825214
65294724	0.670180
37799818	0.714444
3006209	0.882729
26667737	0.771064
2998244	0.767035
40740584	0.654112
139844061	0.703779
121084573	0.899203
35053880	0.895718
139239743	0.974706
15621535	0.934263
86994564	0.620066
42335904	0.669532
60945840	0.655512
116575231	0.870985
42408590	0.922397
31579833	0.907625
141605524	1.300336
14188882	0.686345
45581605	0.813904
33944212	0.814978
44464435	0.769718
18792661	0.928492
42158081	0.784509
44565653	1.047864
11415937	1.030913
39525025	0.974481
38237560	1.679258
40102233	0.654941
45270693	0.832122
46179105	0.982690
41565171	0.926270
64514421	1.091352
141668875	0.714857
141023685	0.779248
141668885	0.829846
1777887	0.730498
14737940	0.602351
28888907	0.999237
30216220	0.661733
5001451	0.886473
656915	0.694899
41923968	0.618910
140222909	0.716302
45244957	0.610294
2829968	0.801414
6374885	0.660907
29096364	0.651616
21494159	0.655662
71043107	0.647130
32660858	0.948580
955331	0.807746
38948647	0.930799
141027578	0.888997
47092844	0.984550
19165719	0.614705
41806352	0.685032
53301583	0.704280
2953061	0.831831
20747008	0.725162
86627197	0.916135
21460567	0.745079
42437153	0.672414
140475370	1.263112
35143870	0.749425
45527403	0.771844
32901032	1.110718
55853233	0.881534
21103574	0.665204
8280589	0.790775
119339452	0.619875
64194458	0.663333
1229561	0.870305
39455327	0.697646
41111643	0.614218
32518191	1.321386
45094884	0.797245
44163723	0.900546
40311569	0.884170
21285018	0.808383
44440853	0.648095
35326651	0.853713
91252173	0.756976
17436439	0.714115
27656134	1.207255
140642762	0.689426
141365022	0.812016
61627760	0.779249
39546280	1.031033
35667668	0.654273
110756280	0.975049
137420430	0.629137
3013776	0.937699
54108693	0.633333
44979583	0.691810
32393396	0.844334
43270251	0.659703
39391576	1.169173
18416878	0.624752
72727033	0.994695
35189222	0.695284
33041592	0.610429
35712104	0.880608
48063240	1.078625
45473813	0.650979
47950690	1.469326
44677626	0.683527
43280471	1.226936
44721495	0.662518
139356493	0.683168
32060816	1.350693
45928021	0.725639
2910534	0.812664
100487487	0.862124
140156019	1.111748
2958236	0.780131
18513019	0.632375
48983300	0.632489
64483589	1.186094
99879156	1.025552
29010256	0.782958
88686469	0.741044
115539057	1.229540
70711878	1.171649
70771773	1.751506
37446336	0.912088
46701954	0.761660
137807408	1.103057
29388198	0.752884
37313122	0.628455
16146327	1.077219
79195099	0.655600
45326099	0.644220
46362617	0.710555
44339567	0.864069
15475236	0.683849
41618651	0.701484
115699397	0.653938
44154419	1.141878
16114337	0.855400
141873755	1.228734
141708545	0.769230
17857014	0.999459
40802506	0.962749
45048958	0.639595
102984117	0.601772
141328605	0.682310
140705803	1.191628
44245494	0.869749
41591078	0.741603
18144652	0.908904
60945893	0.712863
64478096	1.071522
46419978	0.868410
141738043	0.854620
42650760	0.620265
42642148	0.698217
128926646	0.741922
47353424	1.647966
20761045	1.025508
6030686	0.685721
15149355	0.830513
18822654	1.195876
2989169	1.018090
61556164	0.887752
141546077	0.863548
140712823	0.941540
40401957	0.710312
1538363	0.715338
141668869	0.738027
18695142	0.849217
61750279	0.676569
61750467	1.210167
46108674	0.904075
16351739	0.615656
34104101	0.649698
125616757	0.766402
47789983	0.729889
38021157	0.771529
84077960	0.605057
140783780	0.934957
33715684	0.778255
71043154	0.866739
14195177	0.812711
29339132	1.048510
46884525	0.705056
44953394	0.930752
47953689	0.691892
39981200	1.053383
42399642	0.700131
114301686	1.265520
16414941	0.786536
36825943	0.909122
141889192	0.988487
39434135	0.659293
141668874	0.976331
86121951	0.668174
47814647	0.607583
47719511	1.295694
6037094	1.330991
36779295	0.605836
41007633	0.867943
141668877	1.237370
4608073	0.764738
71126227	1.014873
46001923	0.844084
48975110	0.747547
2921782	1.085134
140179187	0.787912
30617762	1.224827
58108922	0.715617
141894968	0.674798
2909434	0.948299
28050837	0.639207
140315616	0.813091
19786034	1.031802
2963885	0.709545
140559519	0.989492
35311469	0.781581
45243803	0.872017
2967821	0.677083
3034877	0.811360
40933460	0.711981
90525077	1.031739
2958482	1.023591
92335677	0.611204
44260988	0.921726
15569982	0.636359
8068564	1.215842
64006585	0.608344
140324625	0.913508
42921495	0.918220
6298420	0.810650
45166215	0.964402
140327090	0.650980
141668888	1.543297
140627806	0.679227
15151069	1.017273
45582088	0.606055
47092723	0.759213
29011852	0.905472
957210	1.296097
15462401	0.988217
132138890	1.028397
15570065	1.440018
89077508	1.417942
140815691	1.352696
23810698	0.789899
64507033	0.873953
32211281	0.773726
127304904	0.721671
140177212	0.716793
14249464	1.158083
14187714	0.701460
15144004	0.606407
35370174	0.993195
39329841	0.710005
11324855	0.619207
41082800	0.708290
3791933	0.909752
64472547	0.952975
89077535	0.772062
141023488	0.921561
37420933	0.986613
39368610	0.612965
65291726	0.606509
72535644	0.620838
35048889	0.871804
35384435	0.712321
6250827	1.314282
24484190	0.780303
21406588	0.613937
44191901	1.079720
46383912	0.787225
2815874	0.853860
29399964	0.884112
15104654	0.881649
79312208	0.625965
40991414	0.888570
47753145	1.198698
46494649	0.707410
42435718	1.057741
33915448	0.795451
47950641	0.606491
45534116	0.631734
42646906	1.516860
52097847	0.821388
2821867	0.801147
29022023	1.173836
3089326	1.832100
141668890	1.414513
114284927	0.695437
2989518	0.726146
137274148	0.611059
44172456	0.770288
31163911	0.734420
44487514	0.972908
2952914	0.786163
141622584	0.600445
140705332	0.821483
42777981	0.669361
26692755	0.919938
45366448	0.648626
43119910	0.788361
140701710	1.912709
18512651	0.812844
15538737	1.205497
41075252	0.607902
73074803	0.808296
40599690	0.969676
14614919	1.021813
64472565	0.733914
47566367	0.661574
34499288	0.603524
15124851	0.673883
117708215	0.740083
126056832	1.139924
87956326	0.841664
15149723	1.262221
15006566	0.764058
911012	0.887189
44614797	0.896261
21554888	0.696494
17218080	0.997411
36776623	0.721588
15418733	0.917987
64734368	0.806249
140643585	0.614549
44618207	0.859319
48065027	0.728727
48899513	1.184253
48899484	1.035983
19674622	0.861863
44770066	0.940952
41572598	0.868319
53305066	0.624215
41600910	0.827930
39586683	1.412282
100395153	0.682402
54956989	0.635956
47821078	0.654468
43808849	0.792589
15664746	1.286787
56767822	0.656320
21108683	1.020620
44243383	0.864927
2985654	1.036143
2828700	0.826125
42595372	0.695331
2909339	0.610886
6148905	0.828226
65254043	1.135200
91774456	0.832567
136826314	0.840511
2985862	0.740574
127300511	1.162084
53306262	1.206135
64194456	0.927548
131385208	0.722712
37227396	0.949563
103373727	0.732653
37038766	1.183511
30571306	0.798170
35570968	1.429880
29689633	1.135660
140713305	0.886706
51870455	0.640700
18203549	0.721131
55254282	0.638817
15543579	0.601941
14274621	0.837555
45310405	0.613484
37446352	1.128063
71621700	1.185101
134030431	0.856118
30692821	0.911490
24424336	0.726227
34364658	0.606132
141584271	1.096953
138789089	0.837497
45530832	0.666135
45401821	0.685089
26146370	0.876712
46416335	0.696992
130379735	0.600785
64219690	0.651698
141017238	0.920085
116766059	0.671203
44403441	0.823936
44191776	0.934856
86132402	0.615229
11399859	0.640261
39029985	1.325079
38291477	1.123415
28744746	0.749761
132655218	0.649532
38695032	0.610998
41229140	1.134246
74473262	0.879778
40007167	0.690626
75599601	0.647404
33793708	0.906113
42899403	0.674348
55252128	0.604933
30673176	0.761440
47753192	0.812843
19674588	1.123729
126056893	0.795784
61306165	0.701817
75609496	0.944121
40893448	1.085937
86137452	1.051536
47355820	0.705200
45091222	0.850018
40893985	1.868458
141952152	0.812539
18271929	0.663097
140783704	0.736467
30568933	0.748055
22223179	0.782854
140315946	0.770500
37795529	2.091851
92442652	0.846893
46187053	0.869193
72531174	0.943550
30465873	1.529127
34104090	0.662159
3009857	0.689858
15210816	0.600092
74310330	0.826094
39233903	0.629079
15572124	0.816885
64495752	0.630212
48074772	0.638660
2857817	1.019248
139557716	1.057349
107082884	1.361015
47841172	0.622947
16334203	0.895328
87950990	0.759578
119338812	0.615013
40970854	0.934634
2966637	0.731326
42462849	0.935230
42400248	0.954487
140474757	0.679943
88851091	0.862614
45671969	0.884525
5983927	0.611347
2957789	1.101952
120837495	0.911597
45512835	0.937494
18720861	0.882009
115377850	0.881048
39413264	0.981644
46435541	0.629054
44109245	0.914222
39634397	0.766560
34156122	0.753151
42505402	0.689231
86119538	0.834728
141668870	0.978828
88669252	0.680243
30697009	0.675551
39301095	0.892612
15211640	0.748257
112613749	1.168714
13529598	0.791197
32218275	0.783659
14727841	0.622175
129080385	0.692079
120966055	0.869803
33152945	1.334589
23208040	0.621064
42323013	1.018618
16762681	0.752657
120604765	0.620187
140506941	1.159739
39281946	0.831157
30701419	0.688939
92912211	0.729388
36377938	0.881288
37207130	0.863992
89771684	0.633119
39262396	0.712114
16334646	0.812108
141112153	0.735122
53767130	0.674078
37921912	1.017154
141741309	0.968503
66246247	1.368563
37893582	0.894315
44031001	0.898786
46264717	0.606260
6062473	0.895637
141305476	0.679560
11648711	0.856275
81482261	1.096044
41845145	0.752057
29044390	0.684032

45790630	0.672432
37050922	1.077716
140896933	0.842123
136830229	1.096892
15287204	1.116637
27578359	0.685050
44902644	0.937976
40625178	0.786389
45602114	1.146210
61750522	0.757200
54191919	0.672514
29558943	0.860986
45649715	1.016600
47602079	0.950754
37447024	0.734399
44582999	0.956358
72996614	0.998101
140431831	1.198080
4900078	0.878330
38547292	0.635816
44870322	0.664411
39981612	0.816490
140936149	1.062110
15416301	1.086180
132138116	0.966569
105819208	1.054228
140773107	1.154154
30670867	1.595004
115377335	0.979627
141352876	0.687526
18688934	1.262854
55870461	0.643432
76411216	0.745229
141844828	1.208371
38635274	0.832348
141844804	1.398996
28861406	0.753011
91086214	0.662229
38788404	0.954309
138531231	0.603635
47513590	0.693808
35152640	0.716461
100264127	0.643628
72024682	0.605700
47841166	0.675811
3014296	0.989610
107091936	0.721352
77616081	0.825491
26692783	0.738125
14873322	0.607056
140627197	0.678729
119340876	1.489722
85517670	1.134342
29879561	0.820938
64226007	0.746459
42457845	1.150583
141889196	0.841465
141889198	0.705403
39301028	0.703435
116765176	1.595299
47877608	1.026091
42747791	1.264766
42824277	1.037279
3019935	1.013484
140831957	0.718971
90889240	0.641904
98640717	0.625017
140583574	1.053056
3113309	1.116052
36772982	0.913000
39094739	1.030293
41596230	0.746389
141658876	0.805586
46408398	1.149375
139246891	0.652883
46473142	0.656114
83719705	0.653384
881499	1.063756
140163503	0.987986
140815612	0.679773
141957269	1.074373
38228046	0.895655
55874600	0.793491
41847838	0.644507
41626148	1.137531
15076661	1.008611
141061313	0.786032
48041164	0.942360
140696858	0.810923
140510585	0.665968
140713277	1.288815
141743166	0.676910
41036891	0.940899
91774283	0.990481
120966047	1.113750
47953893	0.610947
141023478	0.705958
46378916	0.643868
16423484	0.627383
39461441	0.685414
132034673	0.829196
30569987	0.624869
80670985	0.719317
2973680	1.014066
43584928	0.904712
64231049	0.920776
2913507	0.962247
117181534	0.750250
72234347	0.839988
140106779	0.979686
21242439	0.709004
74360918	0.698907
140476081	0.659172
141375479	0.637011
80126405	0.679464
43130785	0.603076
63762914	0.704765
44337022	1.449574
45371075	0.732194
3181986	0.746891
33841074	0.602145
1696689	0.658526
29589219	0.903862
2986006	1.021012
132138895	0.964836
87876334	0.686592
37818138	1.371267
31120118	1.650909
91774710	0.605018
45345416	0.751352
858078	0.757220
141225292	0.766687
45466492	0.662145
40426216	1.056909
131647804	0.618559
140051979	0.707733
125619899	0.796280
140107528	0.761359
32159988	0.602181
50197073	0.703135
13655030	0.703154
180335	1.078179
42067249	0.690475
3833477	0.665481
140951354	0.761065
42435561	1.245665
33139458	0.681586
3081424	0.934805
137644933	1.131751
18853326	0.605735
50501057	0.769630
33716368	1.244886
88866781	0.926093
139398075	0.716093
41889312	0.643908
101887166	0.779718
39990581	0.848711
37201849	0.874015
13714648	0.794554
117050695	0.706100
141116089	0.858721
91774194	0.976215
92335671	0.655695
25444754	0.720580
141086786	0.614242
137922446	0.771225
26690289	0.789794
47953802	0.878909
53839213	0.855270
98608991	1.107812
5632753	0.693314
44402964	0.675804
44455561	0.754453
37361040	0.966626
47919145	0.963068
53232826	0.748995
113926571	0.733929
18786721	1.148663
140498238	0.748239
134849532	0.798904
34247356	0.905120
41923502	0.835842
39295782	0.625530
2858389	1.001496
141619012	0.837048
37818134	0.753543
39524466	1.323573
73969553	0.796906
44599445	0.717457
24478652	0.662691
2826023	0.947642
2945946	0.731314
141328604	0.963328
30625607	1.261605
140920273	0.661382
44252501	1.012701
14220860	0.718043
14239367	0.869742
73198349	0.757267
4139953	1.088502
44733566	0.725443
139391138	0.658394
120835184	0.670058
61754882	0.681833
43135979	1.023878
136826430	0.924591
36725775	0.727700
124313957	0.604406
40934747	0.924107
17670900	0.641741
41132867	0.616155
47947743	0.648600
30404128	0.623365
16202906	0.782559
115534535	0.772519
2967221	0.618253
32159952	1.022287
44267560	0.616798
141952365	0.887074
48066864	0.768085
25603873	1.001125
82384317	0.826063
141652935	0.787019
27655909	0.929895
72270972	0.786499
33791388	0.983812
15147391	0.737358
15157501	0.781649
84077571	0.728586
99799273	0.702584
45258008	0.917034
139247161	0.705562
37877755	0.809434
113931365	0.813074
36810396	0.694157
42557400	1.031501
141730733	0.876798
120840095	1.363025
13501847	0.647353
53842177	0.791334
117279256	0.827318
14090860	1.035697
19786663	0.667147
16901171	0.623318
142011719	0.886911
43894273	0.632071
46296904	1.065056
15803834	1.161447
89077150	1.370893
46184743	0.775961
41210573	0.768105
3036294	1.018424
85862546	0.708524
141819189	0.681503
936749	0.693325
5705704	0.727626
140076384	0.613346
14688989	0.925016
30617768	1.041293
28801692	1.085430
18258691	1.079312
28841699	0.630255
35138036	0.680114
43871156	0.651627
43907561	1.102652
91057479	0.731204
33793673	0.630429
30624092	0.832174
140506214	0.798036
46320027	0.641250
104281957	0.743150
13400721	0.805530
89771775	1.092538
45233745	0.825750
3097527	0.680553
126553844	0.941224
46499593	0.656370
43096659	0.708304
42525582	0.687397
100092545	0.747289
84077938	0.994958
38519090	0.687439
41082813	1.165795
29016816	0.609534
32471664	0.640509
22787618	0.897709
2968890	1.381159
98621616	0.975885
16904497	0.626291
44669314	1.027717
140759167	0.753695
15971247	0.787322
33851273	0.649756
17850540	1.188305
2960108	1.007043
45069134	0.617905
15713261	0.838761
89597429	1.038477
45324658	1.121246
2913619	0.666474
21316648	0.885358
33796883	0.619966
98615889	1.016523
141113379	0.601702
16014614	0.880820
45526451	0.832277
140830199	0.782087
81449721	0.610560
29374323	0.984870
2958570	1.278795
32790397	0.602204
4627347	0.861935
14307584	0.677368
3481675	0.612901
42486375	0.961337
29042960	0.703286
18499233	0.687432
141888658	1.457289
64514470	1.104598
141736577	1.082708
32276256	0.625036
141622593	0.703058
115098272	1.154229
118008934	0.980750
85293336	1.188799
91775323	1.111709
44564389	1.016373
44198024	1.115381
139580551	0.795876
32237554	0.685772
62179168	0.625564
15301379	0.692132
140793950	0.664306
57112246	0.871864
36819137	0.829206
140741594	0.708135
29854032	0.608321
140919333	1.190684
45244896	0.883970
38840243	0.713047
46469543	0.824338
120599167	0.703367
47769834	0.809802
140979470	1.248620
38946696	0.676604
118013772	0.814648
33985220	0.655249
8067455	0.775818
13054054	0.766607
26189898	0.980686
46185101	0.635870
141273867	1.095794
43921249	0.624527
83061247	0.689550
73648377	0.621156
55302429	1.096056
37313355	0.768723
39163048	0.713641
73820965	0.723772
31426056	0.793666
139569828	0.750180
6016134	0.607638
139973820	0.693863
41093620	0.852019
45395599	0.970541
21279280	0.851630
139133706	0.654074
50501048	1.035274
44581114	1.243991
126272568	1.034673
132138561	0.763079
45581859	0.665296
1612630	0.731574
33796942	0.880027
18496314	0.602406
47929345	1.308048
65298566	1.386971
3151584	0.690728
47722496	0.973366
140399585	0.856864
82601374	0.666813
42891685	0.919743
45395085	0.657753
14491150	0.613312
66250369	0.972163
44955420	0.749786
73314127	1.179753
141023220	0.703458
15407759	0.664674
130962759	0.779203
141968810	0.910344
17534377	0.648058
42908684	0.831408
124313640	0.856241
47661714	0.712349
35138715	0.693079
15537769	0.689214
20861204	1.780734
141948901	0.602157
40368114	0.821688
33133870	0.673377
20434499	0.696548
141948991	1.392179
45165277	0.943510
134810046	0.690304
30608347	0.629779
141272659	1.283803
73149113	0.923236
85239484	0.820296
33369675	0.976091
38972995	0.692783
2949940	0.819974
84590339	0.608571
15342107	0.981503
89260330	0.778004
36029094	0.733194
93278111	0.808039
140476024	0.646389
14844375	0.751744
48063943	0.930400
140773981	0.829541
2833284	0.702982
60904478	0.795793
90746260	1.214873
142011729	0.852767
43348625	0.602233
6734992	1.235239
21227250	1.302331
67201653	0.896417
17469963	1.055556
140892582	0.704567
15393638	0.664149
51888904	0.934885
46704295	1.027299
14379675	0.750995
45379772	1.373130
39525015	0.673450
113916795	1.600761
2948571	0.611264
43909215	0.712100
142028678	1.028765
35371118	0.663470
17364849	0.662342
41847871	0.795056
47949819	0.763030
115377746	1.161825
29134897	0.824708
4670433	0.610047
33854940	0.980631
142032219	2.026905
32634295	1.065041
32151959	0.727683
48474	0.663466
18775932	0.784802
73314355	0.682723
1839401	0.731868
64219704	0.687344
6105842	0.614901
13717716	0.822586
90911644	0.949759
29042953	0.618369
15279862	0.715552
6921988	0.667939
2980998	0.889468
42848547	0.724952
44349320	0.818735
44287289	0.758008
938537	0.722987
15119113	0.741348
112389401	1.054616
35327611	0.844496
141292923	0.723401
130547239	0.900038
64483484	0.663835
42116490	0.810881
36056321	0.921439
3062375	0.950709
18690264	1.062064
140024972	0.654719
46150054	0.779140
43316800	0.994639
66248182	0.917937
140875116	0.618621
15645353	0.693776
64102800	0.698488
16207119	0.817433
44260407	0.842847
37446291	0.604866
82451864	0.821554
123463533	1.012146
25237276	0.960409
15480250	0.719334
17697096	0.746565
136410967	1.173383
129076136	0.697367
3052671	0.963481
17943944	0.690134
141001992	1.177745
7787006	0.673543
140681980	0.947188
31533089	0.603532
105817435	0.840319
41776469	1.081156
14272263	0.618098
34001217	0.870055
81344918	0.698846
38883246	0.763424
142036695	0.618599
30611632	0.605478
116564747	0.650785
113940031	0.812088
135726892	0.603561
135727147	0.607585
140772855	1.327189
38573831	0.633700
64400091	0.698207
47533068	0.771268
141668306	0.653948
141342823	1.176601
2979930	0.768603
26003024	0.789974
39300998	0.918958
18785506	0.733483
37880160	1.201302
42459182	1.632441
15312199	0.630450
71318622	1.494487
17607053	0.621260
139271220	0.687396
56201524	0.705783
141546420	0.935597
140155077	0.921673
83728537	1.241916
45507821	0.835963
41065561	0.695513
15520677	0.649793
140772766	1.164890
140563239	0.935515
45639151	0.704717
23897597	0.914944
62427538	1.057837
103189348	0.617546
15255376	1.225557
21532752	0.693023
141630303	0.719041
35487786	0.736195
45217355	0.617856
38027801	0.619061
42508366	1.135034
15437127	0.718916
139610379	0.703639
127306622	0.665386
42240065	0.829971
65245834	0.602569
849968	0.783456
127982817	0.818973
140506258	1.270597
47661715	0.866693
141968795	1.212762
141311541	0.611836
8234719	0.861734
139447893	0.839089
26969319	0.716917
43606020	0.785778
29100992	1.604815
141606176	1.068825
142081073	0.725265
42030607	0.782768
35327061	1.292455
43396246	0.985738
141106200	0.981571
44084234	0.887402
43552133	1.044151
44431855	0.668514
6275108	0.632111
7836535	0.670570
3317920	0.613145
44892467	0.620732
17442446	0.652117
86354231	0.930765
18246404	0.784486
77370903	0.612960
142100055	0.609278
30847529	0.652735
3354339	1.129803
127444916	0.641043
99799264	0.738191
39259818	0.965432
46782942	0.800782
17941421	0.699706
141294104	0.943444
140475757	0.711692
39927485	1.208170
33756450	0.813763
131736435	0.948173
3098215	0.602893
140838642	0.914735
141342827	0.801659
39204333	0.785786
141961361	0.746895
41969548	0.683959
89596747	0.794385
15027978	0.711518
14357387	0.657117
34303614	0.693827
141352945	0.672158
4673761	0.606122
19251053	0.895934
142125509	1.935348
42438989	0.689817
141553465	0.938096
13700958	0.853496
39906902	0.639566
5724226	1.029585
14732204	0.698713
141843055	1.135028
13995850	1.072993
45290444	0.655318
2993705	0.871829
141148224	0.975643
23693001	0.624794
48066791	0.929653
35493615	0.690995
47870879	0.677770
116765639	0.706565
28843738	0.928153
40200541	0.634144
141726773	1.169158
42740686	0.832512
5604835	0.704059
34813568	0.782494
55874800	0.621215
54108578	0.625312
18765803	0.777383
40202921	0.860314
140043639	0.776151
140044224	0.865599
141305495	0.640427
42433394	0.827550
103347485	0.647257
141403529	0.746450
2922105	0.880222
20747284	0.619416
25996548	0.908327
141025318	1.046133
45345397	1.046658
26885008	0.913494
40771545	0.679726
87412689	0.829732
87956887	0.816309
36370239	0.638869
140780781	0.987760
97093853	1.530581
33999012	0.726059
28715721	0.812768
13875317	0.729963
141150497	0.896235
90167283	1.213649
45154699	0.716168
16138538	0.615199
141653161	0.797822
93580332	0.805929
141300557	0.782409
141703588	1.397986
92336300	0.907735
92336304	0.997041
140832402	0.930338
60729309	0.854215
16513258	0.619547
92336310	0.611892
87484114	0.628712
36502944	0.652415
39455325	0.646348
141364930	0.608018
40788872	0.632128
3563628	0.676760
47056578	0.929508
2843590	0.657952
39990270	0.872059
45447640	0.724201
46065516	0.799096
122412550	1.081087
137807430	0.689993
88972920	0.810114
23763617	0.874653
44389221	0.803459
45268932	0.851375
126553973	0.906479
141741118	1.456782
60733520	0.747375
141958914	0.706302
36516624	0.663654
45310401	0.743359
139226673	0.703985
14268308	0.630944
57002314	0.612816
28110288	0.761028
44068374	0.791511
6585265	0.766691
33178981	0.761573
91086180	0.710086
42459193	0.647985
17769194	0.819442
4172605	0.834310
14730980	0.740953
33982242	0.791657
39644569	0.889146
33812942	0.617967
141145684	1.253425
141747614	0.616510
35140670	0.609884
15803825	0.934629
65273814	0.628431
141009500	0.946140
16896293	0.661151
14861789	0.825179
142036901	0.754267
35349034	0.610296
44260573	0.807741
17264417	0.766868
28844932	0.911603
46396549	0.637455
46055837	0.728029
47937481	0.602040
32067447	0.639854
45244456	0.817605
43059162	0.617416
91251776	0.680354
32062339	0.972160
141631815	1.112774
142177258	0.704575
139495798	0.717208
44243143	0.628985
90524070	1.035888
28687589	0.603339
90524899	1.073228
28852913	0.703744
32520006	0.650087
142182470	0.666143
58108923	1.179120
35469122	0.745858
45372339	0.882596
15457861	0.636962
41771938	0.607090
140872357	0.702218
16475080	0.656082
43056360	0.718755
758873	0.693395
140327618	0.873483
34832863	0.652946
26996171	0.653645
41785736	0.691773
93500529	0.700860
142449795	1.083105

35344681	1.129221
14498229	0.757586
32255988	0.759134
44870060	1.030195
35668304	0.654859
57445747	0.699143
29315734	0.661487
43389321	0.643104
39934371	0.785605
44564419	0.916900
39109354	0.689807
38241832	0.768804
98611129	0.733961
36017116	0.825072
141712517	1.581745
21279383	0.845349
142149648	1.155542
141919875	0.686661
31074694	0.657336
33598045	0.660484
131384333	0.798424
130176548	0.610240
30666035	0.654940
38910761	0.667267
36648670	0.608661
142474432	0.673785
30491747	0.604908
28801656	0.627073
62427117	0.601874
44336802	0.713984
141890323	0.686282
4420782	0.710778
53749679	0.836901
142454543	1.134648
5775860	0.692388
29130585	0.713260
64484195	0.849000
64514466	0.775747
28113307	0.632638
32188700	0.662492
3837643	0.738728
45119454	0.654565
16332862	0.704528
42523986	0.639591
41165757	0.696399
30656721	0.692821
141150439	1.065080
2868287	0.754341
41572157	0.615849


Recommentations for user 31460292
90057363	0.979922
48779055	1.219488
92336287	0.619047
47833922	0.841020
121090385	1.023198
138531879	1.583254
140554978	2.731774
5689172	1.688485
100540639	1.358565
139241288	1.292006
47878416	0.979951
64190301	0.945828
43972255	0.758818
138704636	2.302911
121623700	0.998329
84741712	1.492488
15053199	0.810409
29020424	1.736118
52928156	0.852643
140701434	1.352199
43972783	1.565849
42649865	0.772957
79409153	2.080463
16504414	1.021562
91960669	1.225156
140594566	1.417287
117276294	1.171021
50874070	1.068287
39188789	1.117866
113966530	1.169820
140772709	1.286793
139447140	1.996504
133631595	1.890002
46361715	0.880706
139270905	1.706772
44126449	1.032962
117501702	1.121705
140317791	2.370730
46324362	1.119576
13838562	1.812879
56306666	0.849693
98598818	1.909992
37944207	0.642718
140059254	0.621110
113945468	1.504893
40934755	1.872431
64481421	1.251896
92031643	1.520630
116765328	1.140778
40324555	0.785648
139566128	1.447769
16828487	1.007742
25207386	1

2965101	1.001677
64195176	1.571292
42221414	1.804996
33990457	0.879664
139218637	0.962030
64438207	1.190539
95227195	0.643711
93306756	0.697059
90323642	0.974599
47756119	0.722661
47501776	1.157788
44857775	0.719969
138327722	2.398575
62388307	0.986068
84077435	0.631267
4203816	1.443529
31148645	2.189646
4142940	0.758540
140193058	0.850779
3004265	1.777510
6598312	0.913431
43835734	0.893237
140538537	1.793686
115539032	0.928330
35193924	1.113079
140189307	1.165949
34180052	1.397278
64551322	0.807019
44589066	0.718125
26689783	0.724347
71133802	1.522020
115854316	1.491717
1846568	0.604116
140723361	1.167110
15557896	0.698958
45367917	0.716328
42785207	1.251399
29880705	0.985735
140509825	1.083571
64529706	0.728910
45244847	0.608490
141293146	0.842783
35468523	1.533107
141293156	0.913022
26324850	1.683258
42785167	0.949692
15714875	1.041606
18681207	0.709828
84741624	1.783936
30653227	0.636675
45099481	0.692384
140076264	1.083934
140559456	1.706146
28846152	1.239531
37438053	1.389474
887

36616186	1.152726
68156205	0.725337
43350527	0.631881
19602079	0.675543
19602017	1.550083
46364198	0.747129
140315195	0.764073
41920104	1.042739
41556602	0.694322
43020973	0.878161
30667082	0.885705
24841564	1.314413
118011573	1.293773
42554991	0.600247
7784464	0.821476
31077184	0.673122
32276503	0.927795
140554980	1.007345
126062817	0.774579
15184791	0.928297
32018369	0.977265
88699160	1.082994
62035806	0.863711
45406483	0.643838
46394565	1.248959
2815823	0.844586
82977002	1.239303
29197107	0.874016
47543141	1.221369
61713583	0.910412
140835072	0.756859
101323725	0.839918
141642931	1.035755
41655744	0.747811
88699179	0.911576
32143758	0.823763
126056857	1.446061
25991673	1.401241
5750760	0.613051
62427943	0.738367
140583181	0.662203
30070312	0.774077
23356420	0.972170
43067900	0.670733
35146152	0.601146
44365604	1.177028
120966039	1.839686
139652671	0.933684
139728506	0.791841
39233926	0.610446
44161533	0.680249
4108761	0.790970
2989942	0.632288
141003957	0.847734
40994320	1.101410
39

39229931	0.654102
2839462	0.611081
133019363	0.854681
3089558	1.325516
15044887	1.276562
45526605	0.747958
41651076	0.624421
90057734	0.865997
16134803	0.758418
40414930	0.619333
16934895	0.611612
43426098	0.709837
44584342	1.163113
64219529	0.603091
16408397	0.718523
45244222	0.983752
48900081	0.892749
32184665	1.061493
141009499	0.708795
14149796	0.995253
39947282	0.601753
6298090	0.634653
115860799	0.618842
39327621	0.787724
52559092	0.993473
91775287	1.063799
46327618	0.827720
45332884	0.784210
90525107	0.864434
117497978	0.668559
89597467	0.808923
47093309	0.738370
2928466	1.348606
42859900	1.222919
45345350	0.847883
16351937	0.906626
45473417	0.832054
30618169	0.728347
47779423	0.690199
27182695	1.075036
4175690	0.730785
40994796	0.977777
140712451	1.246644
9189233	0.664205
77901397	0.773427
17574428	0.665670
50257416	0.732398
15588710	0.664496
124028568	1.023795
14063223	0.699724
3342002	0.797728
63890475	1.028914
139125083	1.452042
21103573	0.967037
44172427	0.827385
120838003	

45790630	0.672432
37050922	1.077716
140896933	0.842123
136830229	1.096892
15287204	1.116637
27578359	0.685050
44902644	0.937976
40625178	0.786389
45602114	1.146210
61750522	0.757200
54191919	0.672514
29558943	0.860986
45649715	1.016600
47602079	0.950754
37447024	0.734399
44582999	0.956358
72996614	0.998101
140431831	1.198080
4900078	0.878330
38547292	0.635816
44870322	0.664411
39981612	0.816490
140936149	1.062110
15416301	1.086180
132138116	0.966569
105819208	1.054228
140773107	1.154154
30670867	1.595004
115377335	0.979627
141352876	0.687526
18688934	1.262854
55870461	0.643432
76411216	0.745229
141844828	1.208371
38635274	0.832348
141844804	1.398996
28861406	0.753011
91086214	0.662229
38788404	0.954309
138531231	0.603635
47513590	0.693808
35152640	0.716461
100264127	0.643628
72024682	0.605700
47841166	0.675811
3014296	0.989610
107091936	0.721352
77616081	0.825491
26692783	0.738125
14873322	0.607056
140627197	0.678729
119340876	1.489722
85517670	1.134342
29879561	0.820938
64226007	0.746

35344681	1.129221
14498229	0.757586
32255988	0.759134
44870060	1.030195
35668304	0.654859
57445747	0.699143
29315734	0.661487
43389321	0.643104
39934371	0.785605
44564419	0.916900
39109354	0.689807
38241832	0.768804
98611129	0.733961
36017116	0.825072
141712517	1.581745
21279383	0.845349
142149648	1.155542
141919875	0.686661
31074694	0.657336
33598045	0.660484
131384333	0.798424
130176548	0.610240
30666035	0.654940
38910761	0.667267
36648670	0.608661
142474432	0.673785
30491747	0.604908
28801656	0.627073
62427117	0.601874
44336802	0.713984
141890323	0.686282
4420782	0.710778
53749679	0.836901
142454543	1.134648
5775860	0.692388
29130585	0.713260
64484195	0.849000
64514466	0.775747
28113307	0.632638
32188700	0.662492
3837643	0.738728
45119454	0.654565
16332862	0.704528
42523986	0.639591
41165757	0.696399
30656721	0.692821
141150439	1.065080
2868287	0.754341
41572157	0.615849
